# Hybrid
Going to attempt a hybrid model after the example of [this Teck Meng Wong notebook](https://www.kaggle.com/teckmengwong/tps2201-hybrid-time-series/notebook).

- 20220122: Going to try to form ensembles, with more code architecture. Forecasting models will include Prophet, NeuralProphet, Ridge, and Linear (with more to come -- e.g. perhaps transformers and other DNNs); residual models will include GBMs, perhaps some tabular DNNs too.

In [1]:
# notebook configuration
# if '/sf/' in pwd:
#     COLAB, SAGE = False, False
# elif 'google.colab' in str(get_ipython()):
#     COLAB, SAGE = True, False # do colab-specific installs later
# else:
#     COLAB, SAGE = False, True
    
CONTEXT = 'local' # or 'colab', 'sage', 'kaggle'
USE_GPU = True 
%config Completer.use_jedi = False

## Imports

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import requests # for telegram notifications
from tqdm.notebook import tqdm

from joblib import dump, load

import datetime as dt

Now, non-stdlib imports

In [3]:
# model selection
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

# metrics
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error

# normalization
# from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, QuantileTransformer
# from gauss_rank_scaler import GaussRankScaler

# feature generation
# import category_encoders as ce

# models -- will be imported JIT
# from catboost import CatBoostClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from sklearn.ensemble import StackingClassifier, RandomForestClassifier
import torch
from torch.optim import Adam, AdamW, Adagrad, SGD, RMSprop, LBFGS
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts, CyclicLR, OneCycleLR, StepLR, CosineAnnealingLR
# from pytorch_widedeep import Trainer
# from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
# from pytorch_widedeep.models import Wide, TabMlp, WideDeep, SAINT#, TabTransformer, TabNet, TabFastFormer, TabResnet
# from pytorch_widedeep.metrics import Accuracy
# from pytorch_widedeep.callbacks import EarlyStopping, LRHistory, ModelCheckpoint

# feature reduction
# from sklearn.decomposition import PCA
# from umap import UMAP

# clustering
# from sklearn.cluster import DBSCAN, KMeans
# import hdbscan

# feature selection
# from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
# import featuretools as ft
# from BorutaShap import BorutaShap
# from boruta import BorutaPy

# tracking 
import wandb
from wandb.xgboost import wandb_callback
from wandb.lightgbm import wandb_callback
os.environ['WANDB_NOTEBOOK_NAME'] = f"nb_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [4]:
# # time series
# import tsfresh

# import darts
# from darts import TimeSeries
# from darts.models import ExponentialSmoothing, AutoARIMA, ARIMA, Prophet, RandomForest, RegressionEnsembleModel, RegressionModel, TFTModel, TCNModel, TransformerModel, NBEATSModel
import holidays
import dateutil.easter as easter
from prophet import Prophet
from neuralprophet import NeuralProphet

## Routing

Now, datapath setup

In [5]:
if CONTEXT == 'colab':
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
    # datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/dec2021/')
    root = Path('') # TODO

elif CONTEXT == 'sage':
    root = Path('') # TODO
    
elif CONTEXT == 'kaggle':
    root = Path('') # TODO
    
else: # if on local machine
    root = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/jan2022/')
    datapath = root/'datasets'
    # edapath = root/'EDA'
    # modelpath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/models/')
    predpath = root/'preds'
    subpath = root/'submissions'
    studypath = root/'studies'
    
    for pth in [datapath, predpath, subpath, studypath]:
        pth.mkdir(exist_ok=True)

## Helpers

In [6]:
SEED = 42

# Function to seed everything but the models
def seed_everything(seed, pytorch=True, reproducible=True):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    if pytorch:
        torch.manual_seed(seed) # set torch CPU seed
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed) # set torch GPU(s) seed(s)
        if reproducible and torch.backends.cudnn.is_available():
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

seed_everything(seed=SEED)

In [7]:
def reduce_memory_usage(df, verbose=True):
    """
    Function to reduce memory usage by downcasting datatypes in a Pandas DataFrame when possible.
    
    h/t to Bryan Arnold (https://www.kaggle.com/puremath86/label-correction-experiments-tps-nov-21)
    """
    
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [8]:
tg_api_token = 'your_api_token' # for Galileo (jupyter_watcher_bot) on Telegram
tg_chat_id = 'your_chat_id'

import requests

def send_tg_message(text='Cell execution completed.'):  
    """
    h/t Ivan Dembicki Jr. for the base version 
    (https://medium.com/@ivan.dembicki.jr/notifications-in-jupyter-notebook-with-telegram-f2e892c55173)
    """
    requests.post('https://api.telegram.org/' +  'bot{}/sendMessage'.format(tg_api_token),
                  params=dict(chat_id=tg_chat_id, text=text))

In [9]:
def SMAPE(y_true, y_pred):
    '''
    h/t Jean-François Puget (@CPMP) -- see https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/36414
    '''
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

In [10]:
# https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/282735
def better_than_median(inputs, axis):
    """Compute the mean of the predictions if there are no outliers,
    or the median if there are outliers.

    Parameter: inputs = ndarray of shape (n_samples, n_folds)"""
    spread = inputs.max(axis=axis) - inputs.min(axis=axis) 
    spread_lim = 0.45
    print(f"Inliers:  {(spread < spread_lim).sum():7} -> compute mean")
    print(f"Outliers: {(spread >= spread_lim).sum():7} -> compute median")
    print(f"Total:    {len(inputs):7}")
    return np.where(spread < spread_lim,
                    np.mean(inputs, axis=axis),
                    np.median(inputs, axis=axis))

In [11]:
# https://www.kaggle.com/teckmengwong/tps2201-hybrid-time-series
def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("1Y") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax

In [12]:
# https://www.kaggle.com/teckmengwong/tps2201-hybrid-time-series
def fourier_features(index, freq, order):
    time = np.arange(len(index), dtype=np.float32)
    k = 2 * np.pi * (1 / freq) * time
    features = {}
    for i in range(1, order + 1):
        features.update({
            f"sin_{freq}_{i}": np.sin(i * k),
            f"cos_{freq}_{i}": np.cos(i * k),
        })
    return pd.DataFrame(features, index=index)

## Dataset Setup

### Original Data Loading

In [13]:
# dataset_params will initially include either trivial class instances or loaded, precomputed artifacts
dataset_params = {
    'train_source': str(datapath/'train.csv'),
    'target_source': str(datapath/'train.csv'),
    'test_source': str(datapath/'test.csv'),
    # 'scaler': str(RobustScaler()),
    # 'pca': str(load(datapath/'pca_mle-RobustScaled_orig_trainset.joblib')),
    # 'umap': str(load(datapath/'umap_reducer-20211107-n_comp10-n_neighbors15-rs42-pca_mle-RobustScaled_orig_trainset.joblib')),
}   

# referring back to the already-entered attributes, specify how the pipeline was sequenced
# dataset_params['preprocessing_pipeline'] = str([dataset_params['scaler'], dataset_params['pca'], dataset_params['umap']]) # ACTUALLY this is unwieldy
# dataset_params['preprocessing_pipeline'] = '[scaler, pca, umap]' # more fragile, but also more readable

# now, load the datasets and generate more metadata from them
train_df = pd.read_csv(datapath/'train.csv')
test_df = pd.read_csv(datapath/'test.csv')
orig_train_df = train_df.copy()
orig_test_df = test_df.copy()

Since the dates are natively `Object` dtype (i.e. strings), we have to convert them:

In [14]:
# https://www.kaggle.com/ambrosm/tpsjan22-03-linear-model
for df in [train_df, test_df]:
    df['date'] = pd.to_datetime(df.date)

# for convenience later
countries = ['Sweden', 'Finland', 'Norway']
stores = ['KaggleMart', 'KaggleRama']
products = ['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker']

Provisionally, I'm going to concatenate together the `train_df` and `test_df` for preprocessing, to avoid having to constantly apply transforms twice (since I don't anticipate doing any transforms that might allow data leakage to occur).

In [15]:
all_df = pd.concat([train_df, test_df], axis=0)
# all_df.columns
print(len(all_df) == len(train_df) + len(test_df))
del train_df, test_df

True


### GDP Data
Here's data from Carl McBride Ellis ([notebook](https://www.kaggle.com/carlmcbrideellis/gdp-of-finland-norway-and-sweden-2015-2019) and [dataset](https://www.kaggle.com/carlmcbrideellis/gdp-20152019-finland-norway-and-sweden) for doing GDP comparisons. They're frequently used in other entries. I've created a function to add them on.

In [16]:
def add_gdp_data(df):
    gdp_df = pd.read_csv(datapath/'GDP_data_2015_to_2019_Finland_Norway_Sweden.csv')
    gdp_df.set_index('year', inplace=True)
    def get_gdp(row):
        country = 'GDP_' + row.country
        return gdp_df.loc[row.date.year, country]

    df['gdp'] = np.log1p(df.apply(get_gdp, axis=1))
    return df

I'll also define here (but perhaps move later) the GDP exponent, which will be used to transform the targets before inference (dividing num_sold by the $GDP^{1.212}$ and then taking the logarithm (after @ambrosm)

In [17]:
gdp_exponent = 1.2121103201489674 # see https://www.kaggle.com/ambrosm/tpsjan22-03-linear-model for an explanation

In [18]:
all_df = add_gdp_data(all_df)

In [19]:
all_df

,row_id,date,country,store,product,num_sold,gdp
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329.0,5.461456
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520.0,5.461456
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146.0,5.461456
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572.0,5.461456
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911.0,5.461456
...,...,...,...,...,...,...,...
6565,32863,2019-12-31,Sweden,KaggleMart,Kaggle Hat,NaN,6.282042
6566,32864,2019-12-31,Sweden,KaggleMart,Kaggle Sticker,NaN,6.282042
6567,32865,2019-12-31,Sweden,KaggleRama,Kaggle Mug,NaN,6.282042
6568,32866,2019-12-31,Sweden,KaggleRama,Kaggle Hat,NaN,6.282042


## Feature Engineering

### Time Features

The goal of this function is to create features that will capture seasonalities -- but **not** trends. The trends will (hopefully) be captured by the deployment of linear forecasting algorithms on raw time series data (consisting exclusively of dates and targets); we want to have seasonalities that the residual models can learn, however -- holidays, weekly patterns, climactic season patterns, etc.

The cell below will generate the `holidays` library's entries for the three countries. I may want to follow the template of @teckmengwong's code below, and add more holidays -- then, do some feature importance checking, and perhaps whittle down the features accordingly.

In [20]:
for c in [holidays.Finland, holidays.Sweden, holidays.Norway]:
#     print(c)
    for h in c(years = [2019], observed=True).items():
#         print(h)
        pass

In [21]:
def temporal_engineering(df):
    '''
    Function inspired by / borrowing from @teckmengwong and @ambrosm to create time features that will
    capture seasonality.
    '''
    
#     df[YEAR] = df[DATE].dt.year
    df['month'] = df['date'].dt.month
#     df['week'] = df['date'].dt.week # not used by Teck Meng Wong
#     df['day'] = df['date'].dt.day # not used by Teck Meng Wong
#     df['day_of_year'] = df['date'].dt.dayofyear # not used by Teck Meng Wong
#     df['day_of_month'] = df['date'].dt.days_in_month # not used by Teck Meng Wong
#     df['day_of_week'] = df['date'].dt.dayofweek # not used by Teck Meng Wong
#    df['weekday'] = df['date'].dt.weekday # not used by Teck Meng Wong
    # Teck Meng Wong mapped the integers to first-letters in triplets
    # I'm leaving it as integers, where winter=1, spring=2, summer=3, fall=4
    df['season'] = ((df['date'].dt.month % 12 + 3) // 3) #.map({1:'DJF', 2: 'MAM', 3:'JJA', 4:'SON'})
#     df['month'] = df['month'].apply(lambda x: calendar.month_abbr[x])

    df['wd4'] = df['date'].dt.weekday == 4
    df['wd56'] = df['date'].dt.weekday >= 5
#     df['wd6'] = df['date'].dt.weekday >= 6
#     df.loc[(df.date.dt.year != 2016) & (df.date.dt.month >=3), 'day_of_year'] += 1 # fix for leap years
    
    # 21 days cyclic for lunar
    dayofyear = df.date.dt.dayofyear # for convenience
    
    # here he's creating Fourier features
    for k in range(1, 32, 4):
        df[f'sin{k}'] = np.sin(dayofyear / 365 * 2 * math.pi * k)
        df[f'cos{k}'] = np.cos(dayofyear / 365 * 2 * math.pi * k)
        df[f'finland_sin{k}'] = np.where(df['country'] == 'Finland', df[f'sin{k}'], 0)
        df[f'finland_cos{k}'] = np.where(df['country'] == 'Finland', df[f'cos{k}'], 0)
        df[f'norway_sin{k}'] = np.where(df['country'] == 'Norway', df[f'sin{k}'], 0)
        df[f'norway_cos{k}'] = np.where(df['country'] == 'Norway', df[f'cos{k}'], 0)
        df[f'store_sin{k}'] = np.where(df['store'] == 'KaggleMart', df[f'sin{k}'], 0)
        df[f'store_cos{k}'] = np.where(df['store'] == 'KaggleMart', df[f'cos{k}'], 0)
        df[f'mug_sin{k}'] = np.where(df['product'] == 'Kaggle Mug', df[f'sin{k}'], 0)
        df[f'mug_cos{k}'] = np.where(df['product'] == 'Kaggle Mug', df[f'cos{k}'], 0)
        df[f'sticker_sin{k}'] = np.where(df['product'] == 'Kaggle Sticker', df[f'sin{k}'], 0)
        df[f'sticker_cos{k}'] = np.where(df['product'] == 'Kaggle Sticker', df[f'cos{k}'], 0)
    
#     df[f'semiweekly_sin'] = np.sin(dayofyear / 365 * 2 * math.pi * 14)
#     df[f'semiweekly_cos'] = np.cos(dayofyear / 365 * 2 * math.pi * 14)
#     df[f'lunar_sin'] = np.sin(dayofyear / 365 * 2 * math.pi * 21)
#     df[f'lunar_cos'] = np.cos(dayofyear / 365 * 2 * math.pi * 21)
    df[f'season_sin'] = np.sin(dayofyear / 365 * 2 * math.pi * 91.5)
    df[f'season_cos'] = np.cos(dayofyear / 365 * 2 * math.pi * 91.5)
#     df = pd.concat([df, pd.DataFrame({f'fin{ptr[1]}':
#                                       (df.date == pd.Timestamp(ptr[0])) & (df.country == 'Finland')
#                                       for ptr in holidays.Finland(years = [2015,2016,2017,2018,2019]).items()})], axis=1)
#     df = pd.concat([df, pd.DataFrame({f'nor{ptr[1]}':
#                                       (df.date == pd.Timestamp(ptr[0])) & (df.country == 'Norway')
#                                       for ptr in holidays.Norway(years = [2015,2016,2017,2018,2019]).items()})], axis=1)
#     df = pd.concat([df, pd.DataFrame({f'swe{ptr[1]}':
#                                       (df.date == pd.Timestamp(ptr[0])) & (df.country == 'Sweden')
#                                       for ptr in holidays.Sweden(years = [2015,2016,2017,2018,2019]).items()})], axis=1)

    # End of year
    # Dec - teckmengwong
    for d in range(24, 32):
        df[f"dec{d}"] = (df.date.dt.month == 12) & (df.date.dt.day == d)
    # I'm unsure of the logic of only doing this for Norway
    for d in range(24, 32):
        df[f"n-dec{d}"] = (df.date.dt.month == 12) & (df.date.dt.day == d) & (df.country == 'Norway')
    
    # not sure why he's using different date ranges for each country here
    # Jan - teckmengwong
    for d in range(1, 14):
        df[f"f-jan{d}"] = (df.date.dt.month == 1) & (df.date.dt.day == d) & (df.country == 'Finland')
    for d in range(1, 10):
        df[f"n-jan{d}"] = (df.date.dt.month == 1) & (df.date.dt.day == d) & (df.country == 'Norway')
    for d in range(1, 15):
        df[f"s-jan{d}"] = (df.date.dt.month == 1) & (df.date.dt.day == d) & (df.country == 'Sweden')
    
    
    # May - tekcmengwong
    for d in list(range(1, 10)): # May Day and after, I guess
        df[f"may{d}"] = (df.date.dt.month == 5) & (df.date.dt.day == d)
    for d in list(range(19, 26)):
        df[f"may{d}"] = (df.date.dt.month == 5) & (df.date.dt.day == d) & (df.country == 'Norway')
    # June 
    for d in list(range(8, 14)):
        df[f"june{d}"] = (df.date.dt.month == 6) & (df.date.dt.day == d) & (df.country == 'Sweden')
    
    #Swedish Rock Concert - teckmengwong
    #Jun 3, 2015 – Jun 6, 2015
    #Jun 8, 2016 – Jun 11, 2016
    #Jun 7, 2017 – Jun 10, 2017
    #Jun 6, 2018 – Jun 10, 2018
    #Jun 5, 2019 – Jun 8, 2019
    swed_rock_fest  = df.date.dt.year.map({2015: pd.Timestamp(('2015-06-6')),
                                         2016: pd.Timestamp(('2016-06-11')),
                                         2017: pd.Timestamp(('2017-06-10')),
                                         2018: pd.Timestamp(('2018-06-10')),
                                         2019: pd.Timestamp(('2019-06-8'))})

    df = pd.concat([df, pd.DataFrame({f"swed_rock_fest{d}":
                                      (df.date - swed_rock_fest == np.timedelta64(d, "D")) & (df.country == 'Sweden')
                                      for d in list(range(-3, 3))})], axis=1)

    
    # Last Wednesday of June - teckmengwong
    wed_june_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-06-24')),
                                         2016: pd.Timestamp(('2016-06-29')),
                                         2017: pd.Timestamp(('2017-06-28')),
                                         2018: pd.Timestamp(('2018-06-27')),
                                         2019: pd.Timestamp(('2019-06-26'))})
    for d in list(range(-4, 6)):
        df[f"wed_june{d}"] = (df.date - wed_june_date == np.timedelta64(d, "D")) & (df.country != 'Norway')
        
    # First Sunday of November - teckmengwong
    sun_nov_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-11-1')),
                                         2016: pd.Timestamp(('2016-11-6')),
                                         2017: pd.Timestamp(('2017-11-5')),
                                         2018: pd.Timestamp(('2018-11-4')),
                                         2019: pd.Timestamp(('2019-11-3'))})
    df = pd.concat([df, pd.DataFrame({f"sun_nov{d}":
                                      (df.date - sun_nov_date == np.timedelta64(d, "D")) & (df.country == 'Norway')
                                      for d in list(range(0, 9))})], axis=1)
    
    # First half of December (Independence Day of Finland, 6th of December) -teckmengwong
    df = pd.concat([df, pd.DataFrame({f"dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d) & (df.country == 'Finland')
                                      for d in list(range(6, 14))})], axis=1)
    
    # Easter -teckmengwong
    easter_date = df.date.apply(lambda date: pd.Timestamp(easter.easter(date.year)))
    df = pd.concat([df, pd.DataFrame({f"easter{d}":
                                      (df.date - easter_date == np.timedelta64(d, "D"))
                                      for d in list(range(-2, 11)) + list(range(40, 48)) + list(range(50, 59))})], axis=1)
    
    return df

In [22]:
temporal_all_df = temporal_engineering(all_df)

In [23]:
temporal_all_df

,row_id,date,country,store,product,num_sold,gdp,month,season,wd4,...,easter47,easter50,easter51,easter52,easter53,easter54,easter55,easter56,easter57,easter58
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,False
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,False
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,False
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,False
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,32863,2019-12-31,Sweden,KaggleMart,Kaggle Hat,NaN,6.282042,12,1,False,...,False,False,False,False,False,False,False,False,False,False
6566,32864,2019-12-31,Sweden,KaggleMart,Kaggle Sticker,NaN,6.282042,12,1,False,...,False,False,False,False,False,False,False,False,False,False
6567,32865,2019-12-31,Sweden,KaggleRama,Kaggle Mug,NaN,6.282042,12,1,False,...,False,False,False,False,False,False,False,False,False,False
6568,32866,2019-12-31,Sweden,KaggleRama,Kaggle Hat,NaN,6.282042,12,1,False,...,False,False,False,False,False,False,False,False,False,False


At this point, the `temporal_all_df` DataFrame contains all the time features for both the training and testing sets.
* **Todo**: consider not only adding in holidays from `holidays`, but also borrowing ideas from the AmbrosM Linear notebook too (which creates fewer features, populating them instead with temporal distances from the selected holidays).

### Target Transformation
Now, I'll do the target transformation proposed by @AmbrosM. (I'll do it to the non-encoded DataFrame too, for testing with Prophet and NeuralProphet later.)

In [24]:
for df in [temporal_all_df]:
    df['target'] = np.log(df['num_sold'] / df['gdp']**gdp_exponent)

In [25]:
# encoded_all_df['target'] = np.log(encoded_all_df['num_sold'] / (encoded_all_df['gdp']**gdp_exponent))

In [26]:
temporal_all_df

,row_id,date,country,store,product,num_sold,gdp,month,season,wd4,...,easter50,easter51,easter52,easter53,easter54,easter55,easter56,easter57,easter58,target
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,3.738239
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,4.196010
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,2.925788
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,4.291321
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,4.756724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,32863,2019-12-31,Sweden,KaggleMart,Kaggle Hat,NaN,6.282042,12,1,False,...,False,False,False,False,False,False,False,False,False,NaN
6566,32864,2019-12-31,Sweden,KaggleMart,Kaggle Sticker,NaN,6.282042,12,1,False,...,False,False,False,False,False,False,False,False,False,NaN
6567,32865,2019-12-31,Sweden,KaggleRama,Kaggle Mug,NaN,6.282042,12,1,False,...,False,False,False,False,False,False,False,False,False,NaN
6568,32866,2019-12-31,Sweden,KaggleRama,Kaggle Hat,NaN,6.282042,12,1,False,...,False,False,False,False,False,False,False,False,False,NaN


### Label Encoding

I'm going to encapsulate this in a function so that it can be invoked just-in-time, in the hopes of avoiding confusions with DataFrames.

In [27]:
def label_encoder(df):
    from sklearn.preprocessing import LabelEncoder
    features = ['country', 'product', 'store']
    le_dict = {feature: LabelEncoder().fit(orig_train_df[feature]) for feature in features}
    enc_df = df.copy()
    for feature in features:
        enc_df[feature] = le_dict[feature].transform(df[feature])
    return le_dict, enc_df

In [28]:
# for key in le_dict.keys():
#     print(f"Values for key {key} are {le_dict[key].inverse_transform(range(len(le_dict[key].values())))}")#"
# print(le_dict['country'].inverse_transform([0,1,2]))
# print(le_dict['product'].inverse_transform([0,1,2]))
# print(le_dict['store'].inverse_transform([0,1]))

```
['Finland' 'Norway' 'Sweden']
['Kaggle Hat' 'Kaggle Mug' 'Kaggle Sticker']
['KaggleMart' 'KaggleRama']
```

Now, we'll do the encoding.

At this point, the `encoded_all_df` can be used -- perhaps with a call to `LabelEncoder.inverse_transform` -- to recover the "original" data when necessary (e.g. for feeding it into Prophet and NeuralProphet)

In [29]:
# encoded_all_df = label_encoder(temporal_all_df)

### Pseudolabeling

I'm not going to try this right now, but I may return to it later -- I note that Teck Meng Wong had some good results with it.

### Data Splitting, Modification

Now that the preprocessing is done, I'm going to split the data back into the train and test sets; then, I'll create a view on the dataframes that omits the year. The year-less dataframes will be suitable for residual learning.

In [30]:
# all_df = encoded_all_df.drop(columns=['num_sold', 'row_id'])
all_df = temporal_all_df.drop(columns=['row_id']) # writing over the previous version of `all_df`

In [31]:
tv_df = all_df[:len(orig_train_df)] # training and validation sets -- still not encoded
test_df = all_df[len(orig_train_df):] # still not encoded


In [32]:
# train_df = encoded_all_df.iloc[np.where(encoded_all_df['date'] < '2019-01-01'), :]
# test_df = encoded_all_df[[np.where(encoded_all_df['date'] > '2018-12-31')]]

# encoded_tv_df = encoded_all_df.drop(columns=['row_id'])[:len(orig_train_df)]
# encoded_test_df = encoded_all_df.drop(columns=['row_id'])[len(orig_train_df):]

# valid_df = tv_df[tv_df['date'] > '2017-12-31']
# train_df = tv_df[tv_df['date'] <= '2017-12-31']

# train_and_valid_residual_df = train_and_valid_df.drop(columns=['date'])
# test_residual_df = test_df.drop(columns=['date'])

# len(valid_df) + len(train_df) == len(tv_df)

# encoded_tv_df

# Training

### Forecasting Models Prep
First, we'll set up functions to handle the training of forecasting models which will discern trends, and which may -- or may not -- yield insights concerning seasonality. While the Scikit-Learn models will be able to share a single trainer function, the Prophet and NeuralProphet models have subtly different expectations of their data, and as such will require separate handling.

In [33]:
from sklearn.linear_model import Ridge, HuberRegressor, LinearRegression, Lasso
from sklearn.neural_network import MLPRegressor
from prophet import Prophet
from neuralprophet import NeuralProphet
# earth? wouldn't install via pip on my machine at first

#### (Preprepared Preds)

The next cell contains code to import already-existing predictions -- but I think it's better to centralize the code that produces them here, and will comment out the import code for now.

In [34]:
# prophet_trainset = load(predpath/'20220121_prophet_baseline_trainset.joblib')

# neural_trainset = load(predpath/'20220121_neuralprophet_baseline_trainset.joblib')
# neural_test_preds = load(predpath/'20220121_neuralprophet_baseline_testset.joblib')

# ridge_tv_preds = load(predpath/'20210121_ridge_baseline_trainset_preds.joblib')
# ridge_test_preds = load(predpath/'20220121_ridge_testset_preds.joblib')

And this cell would handle the parsing

In [35]:
# neural_tv_preds = neural_trainset['prophet_forecast']
# prophet_tv_preds = prophet_trainset['prophet_forecast']

# neural_train_preds = neural_tv_preds[:train_length]
# neural_valid_preds = neural_tv_preds[train_length:]

# prophet_train_preds = prophet_tv_preds[:train_length]
# prophet_valid_preds = prophet_tv_preds[train_length:]

# train_length = len(neural_trainset[neural_trainset['date'] <= '2017-12-31'])

# ridge_train_preds = ridge_tv_preds[:train_length]
# ridge_valid_preds = ridge_tv_preds[train_length:]

#### Scikit-Learn Linear Models Prep

Linear models from Scikit-Learn seemingly require that datetime data be converted to numerics.

In [36]:
# train_linear_df = train_df.copy()
# valid_linear_df = valid_df.copy()
# test_linear_df = test_df.copy()
# tv_linear_df = tv_df.copy()



### Forecasters

#### Hyperparameters
I'll hard-code them for now, but in the future may Optuna them. May want to create a dict of all the kwargs to be used for all the models, with the model names as keys

In [37]:
prophet_kwargs = {
    'growth':'linear',
#     'holidays':holidays_train, # will add this in-function
    'n_changepoints':10,
    'changepoint_range':0.4,
    'yearly_seasonality':True,
    'weekly_seasonality':True,
    'daily_seasonality':False,
    'seasonality_mode':'additive',
    'seasonality_prior_scale':25,
    'holidays_prior_scale':100,
    'changepoint_prior_scale':0.01,
    'interval_width':0.5,
    'uncertainty_samples':False
}

neuralprophet_kwargs = {
    'growth':'linear',
    'n_changepoints':10,
    'changepoints_range':0.4,
    'trend_reg':1,
    'trend_reg_threshold':False,
    'yearly_seasonality':True,
    'weekly_seasonality':True,
    'daily_seasonality':False,
    'seasonality_mode':'additive',
    'seasonality_reg':1,
    'n_forecasts':365,
    'normalize':'off'
}

# model_params['hyperparams'] = str(neuralprophet_kwargs)
# model_params['holiday_source'] = 'Prophet builtin for each country'

#### Trainers

##### NeuralProphet
I'm leaving the folds as they are. ~~Label encoding shouldn't matter -- the values are just being iterated over anyway.~~ It does matter because the Prophets use the strings to identify countries' holidays to add. Not sure about doing the target transform -- if you try it, just have the trainer call pass `target='target'`.

In [38]:
prophet_folds = [
    ('2015-01-01', '2018-01-01'),
    ('2018-01-01', '2019-01-01'),
]

In [39]:
# prophet_tv_df = tv_df_encoded.copy() # encoded_tv_df.copy()
# prophet_test_df = test_df_encoded.copy() # encoded_test_df.copy()

In [40]:
# for feature in ['country', 'product', 'store']:
#     prophet_tv_df[feature] = orig_train_df[feature]
#     prophet_test_df[feature] = orig_test_df[feature]


In [41]:
# prophet_tv_df.head()

In [42]:
# countries_enc = le_dict['country'].transform(countries)
# stores_enc = le_dict['store'].transform(stores)
# products_enc = le_dict['product'].transform(products)

# countries, countries_enc

In [43]:
def neuralprophet_trainer(model_kwargs=neuralprophet_kwargs, countries=countries, stores=stores, products=products, folds=prophet_folds, 
                          tv_df=tv_df, test_df=test_df,
#                           df_train=tv_df, df_test=test_df, 
                          target='num_sold', wandb_tracked=False):
    train_smape = 0
    val_smape = 0
    
    # create local versions of the dataframes, to avoid mutation
    df_train = tv_df.copy()
    df_test = test_df.copy()
    
    if wandb_tracked:
#         exmodel_config['arch'] = arch
#         exmodel_config[f'{arch}_params'] = str(model_params)
        wandb.init(
            project="202201_Kaggle_tabular_playground",
            save_code=True,
            tags=wandb_config['tags'],
            name=wandb_config['name'],
            notes=wandb_config['notes'],
            config=exmodel_config
    )
    
    # no label encoding here -- but test it with too
    for country in countries:
        for store in stores:
            for product in products:
                for fold, (start, end) in enumerate(folds):
                    # Skip iteration if it's the last fold
                    if fold == len(folds) - 1:
                        continue

                    # put only those rows in that are in the training window and have the correct country, store, and product
                    train_idx = (df_train['date'] >= start) &\
                                (df_train['date'] < end) &\
                                (df_train['country'] == country) &\
                                (df_train['store'] == store) &\
                                (df_train['product'] == product)

                    # redefine the training set in the local (holdout) sense
                    train = df_train.loc[train_idx, ['date', target]].reset_index(drop=True)

                    val_idx = (df_train['date'] >= folds[fold + 1][0]) &\
                              (df_train['date'] < folds[fold + 1][1]) &\
                              (df_train['country'] == country) &\
                              (df_train['store'] == store) &\
                              (df_train['product'] == product)

                    val = df_train.loc[val_idx, ['date', target]].reset_index(drop=True)

                    # rename the columns for standardization (this seems conventional)
                    train = train.rename(columns={'date': 'ds', target: 'y'})
                    val = val.rename(columns={'date': 'ds', target: 'y'})

#                     model = Prophet(**prophet_kwargs)
                    model = NeuralProphet(**model_kwargs)

                    model = model.add_country_holidays(country_name=country) # uses FacebookProphet or NeuralProphet API to add holidays
                    print(train.columns)
                    model.fit(train, freq='D') # neuralprophet
                    # prophet
#                     train_predictions = model.predict(train[['ds']])['yhat']
#                     val_predictions = model.predict(val[['ds']])['yhat']
                    # neuralprophet
                    train_predictions = model.predict(train)['yhat1']
                    val_predictions = model.predict(val)['yhat1']
                    df_train.loc[train_idx, 'neuralprophet_forecast'] = train_predictions.values
                    df_train.loc[val_idx, 'neuralprophet_forecast'] =  val_predictions.values

                    train_score = SMAPE(train['y'].values, train_predictions.values)
                    val_score = SMAPE(val['y'].values, val_predictions.values)
            
                    if wandb_tracked:
                        wandb.log({f"{(country,store,product)}_valid_smape": val_score})
            
                    train_smape += train_score
                    val_smape += val_score
            
                    print(f'\nTraining Range [{start}, {end}) - {country} - {store} - {product} - Train SMAPE: {train_score:4f}')
                    print(f'Validation Range [{folds[fold + 1][0]}, {folds[fold + 1][1]}) - {country} - {store} - {product} - Validation SMAPE: {val_score:4f}\n')

                    test_idx = (df_test['country'] == country) &\
                               (df_test['store'] == store) &\
                               (df_test['product'] == product)
                    test = df_test.loc[test_idx, ['date']].reset_index(drop=True)
                    
                    test = test.rename(columns={'date': 'ds'})
                    test['y'] = np.nan
                    test_predictions = model.predict(test)['yhat1']
                    
                    
                    df_test.loc[test_idx, 'neuralprophet_forecast'] = test_predictions.values
    
    train_smape /= (3*2*3)
    val_smape /= (3*2*3)
#     train_
    
    if wandb_tracked:
        wandb.log({'overall_train_smape': train_smape, 'overall_valid_smape': val_smape})
        wandb.finish()
    return df_train['neuralprophet_forecast'], df_test['neuralprophet_forecast']#, train_smape, val_smape

##### Prophet Trainer

In [44]:
def prophet_trainer(prophet_kwargs=prophet_kwargs, countries=countries, stores=stores, products=products, folds=prophet_folds, 
                    tv_df=tv_df, test_df=test_df,
#                           df_train=tv_df, df_test=test_df, 
                    target='num_sold', wandb_tracked=False):
    train_smape = 0
    val_smape = 0
    
    # create local versions of the dataframes, to avoid mutation
    df_train = tv_df.copy()
    df_test = test_df.copy()
    
    if wandb_tracked:
#         exmodel_config['arch'] = arch
#         exmodel_config[f'{arch}_params'] = str(model_params)
        wandb.init(
            project="202201_Kaggle_tabular_playground",
            save_code=True,
            tags=wandb_config['tags'],
            name=wandb_config['name'],
            notes=wandb_config['notes'],
            config=exmodel_config
    )
    
    for country in countries:
        for store in stores:
            for product in products:
                for fold, (start, end) in enumerate(folds):
                    # Skip iteration if it's the last fold
                    if fold == len(folds) - 1:
                        continue

                    # put only those rows in that are in the training window and have the correct country, store, and product
                    train_idx = (df_train['date'] >= start) &\
                                (df_train['date'] < end) &\
                                (df_train['country'] == country) &\
                                (df_train['store'] == store) &\
                                (df_train['product'] == product)
                    
#                     print(train_idx)

                    # redefine the training set in the local (holdout) sense
                    train = df_train.loc[train_idx, ['date', target]].reset_index(drop=True)
#                     print(train.shape)

                    val_idx = (df_train['date'] >= folds[fold + 1][0]) &\
                              (df_train['date'] < folds[fold + 1][1]) &\
                              (df_train['country'] == country) &\
                              (df_train['store'] == store) &\
                              (df_train['product'] == product)

                    val = df_train.loc[val_idx, ['date', target]].reset_index(drop=True)

                    # rename the columns for standardization (this seems conventional)
                    train = train.rename(columns={'date': 'ds', target: 'y'})
                    val = val.rename(columns={'date': 'ds', target: 'y'})

                    model = Prophet(**prophet_kwargs)

                    model.add_country_holidays(country_name=country) # uses FacebookProphet API to add holidays
                    model.fit(train)
        
                    train_predictions = model.predict(train[['ds']])['yhat']
                    val_predictions = model.predict(val[['ds']])['yhat']
                    df_train.loc[train_idx, 'prophet_forecast'] = train_predictions.values
                    df_train.loc[val_idx, 'prophet_forecast'] =  val_predictions.values

                    train_score = SMAPE(train['y'].values, train_predictions.values)
                    val_score = SMAPE(val['y'].values, val_predictions.values)
            
                    if wandb_tracked:
                        wandb.log({f"{(country,store,product)}_valid_smape": val_score})
            
                    train_smape += train_score
                    val_smape += val_score
            
                    print(f'\nTraining Range [{start}, {end}) - {country} - {store} - {product} - Train SMAPE: {train_score:4f}')
                    print(f'Validation Range [{folds[fold + 1][0]}, {folds[fold + 1][1]}) - {country} - {store} - {product} - Validation SMAPE: {val_score:4f}\n')

                    test_idx = (df_test['country'] == country) &\
                               (df_test['store'] == store) &\
                               (df_test['product'] == product)
                    test = df_test.loc[test_idx, ['date']].reset_index(drop=True)
                    
                    test = test.rename(columns={'date': 'ds'})
                    test_predictions = model.predict(test[['ds']])['yhat']
                    
                    
                    df_test.loc[test_idx, 'prophet_forecast'] = test_predictions.values
    
    train_smape /= (3*2*3)
    val_smape /= (3*2*3)
    
    if wandb_tracked:
        wandb.log({'overall_train_smape': train_smape, 'overall_valid_smape': val_smape})
        wandb.finish()
    return df_train['prophet_forecast'], df_test['prophet_forecast']#, train_smape, val_smape

##### Scikit-Learn Models

In [45]:
def sklearn_trainer(estimator, model_kwargs={}, tv_df=tv_df, test_df=test_df, #X=X, y=y, X_valid=X_valid, y_valid=y_valid, X_test=X_test, 
                    folds=prophet_folds, countries=countries, stores=stores, products=products, target='target',
                    by_combo=False, wandb_tracked=False):
    
    # create local versions of the dataframes, to avoid mutation
    df_train = tv_df.copy()
    df_test = test_df.copy()
    
    # apply label encoding (which Scikit-Learn models require, but *Prophets don't)
    le_dict, tv_df = label_encoder(df_train) # should leave broader scope's tv_df alone
    _, test_df = label_encoder(df_test) # should leave broader scope's test_df alone
    del df_train, df_test
    
    # encode the lists of countries, stores, and products
    countries = le_dict['country'].transform(countries)
    stores = le_dict['store'].transform(stores)
    products = le_dict['product'].transform(products)
    
    train_smape = 0
    val_smape = 0
    
    if wandb_tracked:
#         exmodel_config['arch'] = arch
#         exmodel_config[f'{arch}_params'] = str(model_params)
        wandb.init(
            project="202201_Kaggle_tabular_playground",
            save_code=True,
            tags=wandb_config['tags'],
            name=wandb_config['name'],
            notes=wandb_config['notes'],
            config=exmodel_config
    )
    
    # drop whichever version of the dependent variable is not being used
#     for df in [tv_df, test_df]:
    if target == 'num_sold': 
        tv_df = tv_df.drop(columns=['target'])
        test_df = test_df.drop(columns=['target'])
    else:
        tv_df = tv_df.drop(columns=['num_sold'])
        test_df = test_df.drop(columns=['num_sold'])
            
#     print("'num_sold' in test_df.columns == ", 'num_sold' in test_df.columns)
    
    if by_combo: # handling each combination of country, store, and product separately
        for country in countries:
            for store in stores:
                for product in products:
                    for fold, (start, end) in enumerate(folds):
                        # Skip iteration if it's the last fold
                        if fold == len(folds) - 1:
                            continue

                        # put only those rows in that are in the training window and have the correct country, store, and product
                        train_idx = (tv_df['date'] >= start) &\
                                    (tv_df['date'] < end) &\
                                    (tv_df['country'] == country) &\
                                    (tv_df['store'] == store) &\
                                    (tv_df['product'] == product)

    #                     print(train_idx)

                        # redefine the training set in the local (holdout) sense
                        train = tv_df.loc[train_idx, :].reset_index(drop=True)
#                         print(train.shape)

                        val_idx = (tv_df['date'] >= folds[fold + 1][0]) &\
                                  (tv_df['date'] < folds[fold + 1][1]) &\
                                  (tv_df['country'] == country) &\
                                  (tv_df['store'] == store) &\
                                  (tv_df['product'] == product)

                        val = tv_df.loc[val_idx, :].reset_index(drop=True)

                        test_idx = (test_df['country'] == country) &\
                                   (test_df['store'] == store) &\
                                   (test_df['product'] == product)
                        test = test_df.loc[test_idx, :].reset_index(drop=True)

                        # with the training and validation sets sorted out, make them integers for model fitting
                        for df in [train, val, test]:
                            df['date'] = df['date'].map(dt.datetime.toordinal)
                        if 'model_forecast' in train.columns:
                            X = train.drop(columns=[target, 'model_forecast'])
                            X_valid = val.drop(columns=[target, 'model_forecast'])
                            X_test = test.drop(columns=[target, 'model_forecast'])
                        else:
                            X = train.drop(columns=[target])
                            X_valid = val.drop(columns=[target])
                            X_test = test.drop(columns=[target])
                        
                        y = train[target]
                        y_valid = val[target]
                        
                        
#                         print(type(X), type(y))
#                         print(f"X has {X.isna().any().sum()} NaNs")
#                         print(f"y has {y.isna().sum()} NaNs")
                        
                        model = estimator(**model_kwargs)
                        model.fit(X,y)

                        model_train_preds = model.predict(X)
                        model_valid_preds = model.predict(X_valid)
                        model_test_preds = model.predict(X_test)

                        tv_df.loc[train_idx, 'model_forecast'] = model_train_preds#.values
                        tv_df.loc[val_idx, 'model_forecast'] =  model_valid_preds#.values
                        test_df.loc[test_idx, 'model_forecast'] = model_test_preds#.values
                    
        
    else: # don't separate out by combination of country, store, and product
        # split the dataset such that 2018 is the validation set
        train = tv_df[tv_df['date'] > '2017-12-31']
        valid = tv_df[tv_df['date'] <= '2017-12-31']
        test = test_df.copy()
#         print(test_df['date'].dtype)
#         print("NaNs after test instantiation: ", test.isna().sum())
    
        # convert the datetime objects to integers and then drop the transformed or untransformed target
            
        for df in [train, valid, test]:
            df['date'] = df['date'].map(dt.datetime.toordinal)

                
        # separate out the independent and dependent variables
        X = train.drop(columns=[target])
        y = train[target]
        X_valid = valid.drop(columns=[target])
        y_valid = valid[target]
        X_test = test.drop(columns=[target])
        
#         print("NaNs after X_test instantiation: ", X_test.isna().sum())
#         print("'num_sold' in X_test.columns == ", 'num_sold' in X_test.columns)
        
#         print(type(X), type(y))
        model = estimator(**model_kwargs)
        model.fit(X,y)
        
        # make predictions on both the training and validation set, and put them together
        model_train_preds = model.predict(X)
        model_valid_preds = model.predict(X_valid)
        model_tv_preds = pd.concat([pd.Series(model_train_preds), pd.Series(model_valid_preds)], axis=0, ignore_index=True)
#         print(len(model_tv_preds) == len(model_train_preds) + len(model_valid_preds))
#         print(f"Length of train preds is {len(model_train_preds)}, length of valid preds is {len(model_valid_preds)}, length of combined is {len(model_tv_preds)}")
        
        # make predictions on the test set
        model_test_preds = model.predict(X_test)
#         print("model_test_preds.shape == ", model_test_preds.shape)
        
#         print(tv_df.info())
    
#         output_tv_df = tv_df.copy()
#         output_test_df = test_df.copy()
#         output_tv_df['model_forecast'] = pd.Series(model_tv_preds)#.values
#         output_test_df['model_forecast'] = pd.Series(model_test_preds)#.values
        tv_df['model_forecast'] = model_tv_preds
        test_df['model_forecast'] = model_test_preds
        
    # reverse the dependent variable transform if appropriate
    if target == 'target':
#             model_tv_preds = np.multiply(np.exp(model_tv_preds), tv_df['gdp']**gdp_exponent)
        tv_df['model_forecast'] = np.exp(tv_df['model_forecast']) * tv_df['gdp']**gdp_exponent
#             output_tv_df['model_forecast'] = np.exp(output_tv_df['model_forecast']) * output_tv_df['gdp']**gdp_exponent

#             model_test_preds = np.multiply(np.exp(model_test_preds), test_df['gdp']**gdp_exponent)
        test_df['model_forecast'] = np.exp(test_df['model_forecast']) * test_df['gdp']**gdp_exponent
#             output_test_df['model_forecast'] = np.exp(output_test_df['model_forecast']) * output_test_df['gdp']**gdp_exponent
#             model_test_preds = np.exp(model_test_preds) * test_df['gdp']**gdp_exponent
        
#         tv_df['model_forecast'] = model_tv_preds
#         test_df['model_forecast'] = model_test_preds
#     return output_tv_df, output_test_df
    return tv_df['model_forecast'], test_df['model_forecast']
#     return tv_df['model_forecast'], test_df['model_forecast']
#     return model_tv_preds, model_test_preds
    

#### Calls

In [46]:
%time
prophet_tv_preds, prophet_test_preds = prophet_trainer(target='num_sold')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs

Training Range [2015-01-01, 2018-01-01) - Sweden - KaggleMart - Kaggle Mug - Train SMAPE: 6.815541
Validation Range [2018-01-01, 2019-01-01) - Sweden - KaggleMart - Kaggle Mug - Validation SMAPE: 8.011073


Training Range [2015-01-01, 2018-01-01) - Sweden - KaggleMart - Kaggle Hat - Train SMAPE: 7.210369
Validation Range [2018-01-01, 2019-01-01) - Sweden - KaggleMart - Kaggle Hat - Validation SMAPE: 7.470548


Training Range [2015-01-01, 2018-01-01) - Sweden - KaggleMart - Kaggle Sticker - Train SMAPE: 7.056273
Validation Range [2018-01-01, 2019-01-01) - Sweden - KaggleMart - Kaggle Sticker - Validation SMAPE: 7.264969


Training Range [2015-01-01, 2018-01-01) - Sweden - KaggleRama - Kaggle Mug - Train SMAPE: 6.716048
Validation Range [2018-01-01, 2019-01-01) - Sweden - KaggleRama - Kaggle Mug - Validation SMAPE: 7.406634


Training Range [2015-01-01, 2018-01-01) - Sweden - KaggleRama - Kaggle Hat - Train SMAPE: 7.000068


In [47]:
%time
# neural_tv_preds, neural_test_preds = neuralprophet_trainer(target='num_sold')
# dump(neural_tv_preds, predpath/'20220124_neuralprophet_tv_preds.joblib')
# dump(neural_test_preds, predpath/'20220124_neuralprophet_test_preds.joblib')
neural_tv_preds = load(predpath/'20220124_neuralprophet_tv_preds.joblib')
neural_test_preds = load(predpath/'20220124_neuralprophet_test_preds.joblib')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.58 µs


In [48]:
# %time
# ridge_tv_preds, ridge_test_preds = sklearn_trainer(estimator=Ridge)

In [49]:
%time
ridge_combo_tv_preds, ridge_combo_test_preds = sklearn_trainer(estimator=Ridge, by_combo=True)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [50]:
# linear_tv_preds, linear_test_preds = sklearn_trainer(estimator=LinearRegression)

In [51]:
linear_combo_tv_preds, linear_combo_test_preds = sklearn_trainer(estimator=LinearRegression, by_combo=True)

In [52]:
huber_combo_tv_preds, huber_combo_test_preds = sklearn_trainer(estimator=HuberRegressor, by_combo=True)

In [53]:
mlp_combo_tv_preds, mlp_combo_test_preds = sklearn_trainer(estimator=MLPRegressor, by_combo=True)

In [54]:
lasso_combo_tv_preds, lasso_combo_test_preds = sklearn_trainer(estimator=Lasso, by_combo=True)

In [55]:
from pyearth import Earth
%time
earth_combo_tv_preds, earth_combo_test_preds = sklearn_trainer(estimator=Earth, by_combo=True)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


In [56]:
earth_combo_tv_preds

0         360.672190
1         584.851806
2         138.401176
3         562.479785
4         944.584508
            ...     
26293     836.947940
26294     254.287204
26295    1058.174059
26296    2036.549357
26297     472.307718
Name: model_forecast, Length: 26298, dtype: float64

In [57]:
y_tv = orig_train_df['num_sold']

In [58]:
for preds in [prophet_tv_preds, ridge_combo_tv_preds, linear_combo_tv_preds, huber_combo_tv_preds, mlp_combo_tv_preds, neural_tv_preds, lasso_combo_tv_preds, earth_combo_tv_preds]:
    print(SMAPE(y_pred=preds, y_true=y_tv))

7.098824805150043
4.781041939812455
4.175573461450916
14.396222549610735
99.04042806269284
7.926662675729486
15.11713989060885
5.338351055481194


Clearly, doing it by combination is the way to go. I think Ridge and LinearRegressor are definitely good to use; Prophet and NeuralProphet are worth including as well. Huber, probably not; MLPRegressor, definitely not barring the discovery of better hyperparams.

### Forecast Bundling
Now, create an iterable collection consisting of all the forecaster predictions.

In [59]:
forecast_tv_preds = {
    'prophet': prophet_tv_preds,
    'neuralprophet': neural_tv_preds,
    'ridge': ridge_combo_tv_preds,
    'linear': linear_combo_tv_preds,
    'huber': huber_combo_tv_preds,
    'lasso': lasso_combo_tv_preds,
    'earth': earth_combo_tv_preds,
}

forecast_test_preds = {
    'prophet': prophet_test_preds,
    'neuralprophet': neural_test_preds,
    'ridge': ridge_combo_test_preds,
    'linear': linear_combo_test_preds,
    'huber': huber_combo_test_preds,
    'lasso': lasso_combo_test_preds,
    'earth': earth_combo_test_preds
}

In [60]:
tv_preds = pd.DataFrame({
    'date': orig_train_df['date'],
    'num_sold': orig_train_df['num_sold'],
    **forecast_tv_preds
})

In [61]:
tv_preds

,date,num_sold,prophet,neuralprophet,ridge,linear,huber,lasso,earth
0,2015-01-01,329,346.560416,329.134521,283.272258,324.188332,239.868861,176.381725,360.672190
1,2015-01-01,520,536.203586,458.008301,439.771734,503.081569,334.618687,334.453055,584.851806
2,2015-01-01,146,143.412803,145.325577,120.885683,136.667932,96.086063,89.802390,138.401176
3,2015-01-01,572,590.117165,552.571167,488.810710,555.531957,330.014790,307.788806,562.479785
4,2015-01-01,911,939.673009,781.967285,771.701702,890.469201,875.922092,583.510103,944.584508
...,...,...,...,...,...,...,...,...,...
26293,2018-12-31,823,898.322121,669.418457,723.714830,846.597603,407.206130,397.285794,836.947940
26294,2018-12-31,250,253.512355,227.158142,205.880079,241.048951,132.620712,126.765608,254.287204
26295,2018-12-31,1004,1039.635205,715.639648,832.192362,975.785339,401.224848,430.368706,1058.174059
26296,2018-12-31,1441,1526.908216,980.234009,1255.885410,1468.776593,863.230649,689.775798,2036.549357


In [62]:
valid_forecast_preds = tv_preds[tv_preds['date'] > '2017-12-31']
valid_forecast_preds

,date,num_sold,prophet,neuralprophet,ridge,linear,huber,lasso,earth
19728,2018-01-01,405,383.365770,374.809906,345.296498,417.388135,249.679981,211.676443,382.162985
19729,2018-01-01,621,602.594228,539.082520,536.708119,637.949409,348.560475,348.732376,609.231013
19730,2018-01-01,176,161.735807,167.919098,147.203184,174.086997,99.898973,107.780206,167.839584
19731,2018-01-01,714,656.201669,635.251648,596.599377,713.788713,343.757663,368.614186,597.369896
19732,2018-01-01,1043,1044.903702,940.582886,930.576635,1110.805235,913.849095,604.430617,1059.300177
...,...,...,...,...,...,...,...,...,...
26293,2018-12-31,823,898.322121,669.418457,723.714830,846.597603,407.206130,397.285794,836.947940
26294,2018-12-31,250,253.512355,227.158142,205.880079,241.048951,132.620712,126.765608,254.287204
26295,2018-12-31,1004,1039.635205,715.639648,832.192362,975.785339,401.224848,430.368706,1058.174059
26296,2018-12-31,1441,1526.908216,980.234009,1255.885410,1468.776593,863.230649,689.775798,2036.549357


In [63]:
valid_forecast_preds.corr()

,num_sold,prophet,neuralprophet,ridge,linear,huber,lasso,earth
num_sold,1.000000,0.972938,0.961440,0.992806,0.995334,0.910517,0.899862,0.984322
prophet,0.972938,1.000000,0.990336,0.985617,0.975158,0.937816,0.939803,0.972174
neuralprophet,0.961440,0.990336,1.000000,0.978596,0.962073,0.947772,0.950331,0.961384
ridge,0.992806,0.985617,0.978596,1.000000,0.996412,0.926965,0.919398,0.989698
linear,0.995334,0.975158,0.962073,0.996412,1.000000,0.910123,0.901341,0.987908
huber,0.910517,0.937816,0.947772,0.926965,0.910123,1.000000,0.922899,0.910400
lasso,0.899862,0.939803,0.950331,0.919398,0.901341,0.922899,1.000000,0.906948
earth,0.984322,0.972174,0.961384,0.989698,0.987908,0.910400,0.906948,1.000000


In [64]:
tv_preds.corr()

,num_sold,prophet,neuralprophet,ridge,linear,huber,lasso,earth
num_sold,1.000000,0.975093,0.960082,0.993110,0.995846,0.907911,0.899352,0.990016
prophet,0.975093,1.000000,0.985693,0.986481,0.977785,0.933401,0.927660,0.974833
neuralprophet,0.960082,0.985693,1.000000,0.978010,0.962507,0.938389,0.939915,0.960241
ridge,0.993110,0.986481,0.978010,1.000000,0.996573,0.926616,0.919336,0.992161
linear,0.995846,0.977785,0.962507,0.996573,1.000000,0.909135,0.900973,0.994045
huber,0.907911,0.933401,0.938389,0.926616,0.909135,1.000000,0.923215,0.907105
lasso,0.899352,0.927660,0.939915,0.919336,0.900973,0.923215,1.000000,0.900047
earth,0.990016,0.974833,0.960241,0.992161,0.994045,0.907105,0.900047,1.000000


So Ridge and Linear perform best, and are quite similar; Prophet and NeuralProphet are next best, and similar to one another; Huber is an outlier (and not so good). Lasso performs worst of all, but is closer to Prophet and NeuralProphet in performance than the others. It's closest of all to NeuralProphet, interestingly.

In [65]:
test_forecast_preds = pd.DataFrame({
    'date': orig_test_df['date'],
#     'num_sold': orig_train_df['num_sold'],
    **forecast_test_preds
})

In [66]:
test_forecast_preds

,date,prophet,neuralprophet,ridge,linear,huber,lasso,earth
0,2019-01-01,402.296576,383.543457,359.817081,392.328390,248.728792,221.118411,417.102143
1,2019-01-01,631.642099,551.044983,560.783324,620.092467,347.337935,347.623611,693.844970
2,2019-01-01,169.992094,173.065674,153.552940,166.788184,99.483226,112.590591,181.642807
3,2019-01-01,684.559821,648.783691,622.237644,675.674104,342.549613,384.790895,594.166542
4,2019-01-01,1092.875362,965.683716,969.065554,1084.980801,911.102831,601.189400,1142.564632
...,...,...,...,...,...,...,...,...
6565,2019-12-31,917.085861,674.891174,742.331990,797.964290,404.876685,391.134492,830.606248
6566,2019-12-31,261.569590,232.082169,210.920805,225.576610,131.774790,129.930008,269.498973
6567,2019-12-31,1065.400121,730.457275,852.091174,925.788409,398.926697,440.077496,1133.560974
6568,2019-12-31,1568.407214,1005.392944,1286.814471,1397.071983,858.595973,677.887045,2627.470943


### Residuals

At this point, I have two DataFrames containing predictions from the forecasting models (which try to learn trends): 
1. `valid_forecast_preds`
2. `test_forecast_preds`

Both still contain the features `'date'` (having a datetime type) and the validation preds contain `num_sold`.

The goal now will be to iteratively generate a final prediction DataFrame containing all possible combinations of the forecasting model predictions and residual predictions.

In [67]:
forecast_models = ['prophet', 'neuralprophet', 'ridge', 'linear', 'huber', 'lasso', 'earth'] # models to use to provide basis for residual predictions

In [68]:
# encoded_all_df

In [69]:
# dump(encoded_all_df, datapath/'encoded_train+testset_with_gdp+teckmengwong-time-features+transformed-target-for-train.joblib')

In [70]:
# hybrid_train_df = encoded_all_df[:len(train_df)]
# hybrid_valid_df = encoded_all_df[len(train_df): len(train_df)+len(valid_df)]

In [71]:
# hybrid_test_df = encoded_all_df[len(train_df)+len(valid_df):]

In [72]:
# hybrid_valid_df

Note that the `tv_df` still contains both transformed targets and `num_sold`.

In [73]:
tv_df

,date,country,store,product,num_sold,gdp,month,season,wd4,wd56,...,easter50,easter51,easter52,easter53,easter54,easter55,easter56,easter57,easter58,target
0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329.0,5.461456,1,1,False,False,...,False,False,False,False,False,False,False,False,False,3.738239
1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520.0,5.461456,1,1,False,False,...,False,False,False,False,False,False,False,False,False,4.196010
2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146.0,5.461456,1,1,False,False,...,False,False,False,False,False,False,False,False,False,2.925788
3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572.0,5.461456,1,1,False,False,...,False,False,False,False,False,False,False,False,False,4.291321
4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911.0,5.461456,1,1,False,False,...,False,False,False,False,False,False,False,False,False,4.756724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26293,2018-12-31,Sweden,KaggleMart,Kaggle Hat,823.0,6.321586,12,1,False,False,...,False,False,False,False,False,False,False,False,False,4.477861
26294,2018-12-31,Sweden,KaggleMart,Kaggle Sticker,250.0,6.321586,12,1,False,False,...,False,False,False,False,False,False,False,False,False,3.286366
26295,2018-12-31,Sweden,KaggleRama,Kaggle Mug,1004.0,6.321586,12,1,False,False,...,False,False,False,False,False,False,False,False,False,4.676652
26296,2018-12-31,Sweden,KaggleRama,Kaggle Hat,1441.0,6.321586,12,1,False,False,...,False,False,False,False,False,False,False,False,False,5.037997


In [74]:
test_df

,date,country,store,product,num_sold,gdp,month,season,wd4,wd56,...,easter50,easter51,easter52,easter53,easter54,easter55,easter56,easter57,easter58,target
0,2019-01-01,Finland,KaggleMart,Kaggle Mug,NaN,5.597614,1,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN
1,2019-01-01,Finland,KaggleMart,Kaggle Hat,NaN,5.597614,1,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN
2,2019-01-01,Finland,KaggleMart,Kaggle Sticker,NaN,5.597614,1,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN
3,2019-01-01,Finland,KaggleRama,Kaggle Mug,NaN,5.597614,1,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN
4,2019-01-01,Finland,KaggleRama,Kaggle Hat,NaN,5.597614,1,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,2019-12-31,Sweden,KaggleMart,Kaggle Hat,NaN,6.282042,12,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN
6566,2019-12-31,Sweden,KaggleMart,Kaggle Sticker,NaN,6.282042,12,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN
6567,2019-12-31,Sweden,KaggleRama,Kaggle Mug,NaN,6.282042,12,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN
6568,2019-12-31,Sweden,KaggleRama,Kaggle Hat,NaN,6.282042,12,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN


One question: should the full `tv_df` be passed as data, or only the validation set? I think it's better to supply as much data as possible, so I'll use the whole thing for now.

In [75]:
from sklearn.model_selection import GroupKFold

In [76]:
len(tv_df)

26298

In [77]:
len(orig_train_df)

26298

In [78]:
from sklearn.metrics import mean_squared_error

In [79]:
import math

#### Residual Trainer 
(assumes Scikit-Learn API)

In [80]:
def residual_trainer(estimator, model_kwargs={}, forecast_models=forecast_models,
                     tv_df=tv_df, test_df=test_df,
                     tv_forecast_preds=tv_preds, test_forecast_preds=test_forecast_preds):
#     df_2018 = tv_df[tv_df['date'] > '2017-12-31']
    # create local versions of the dataframes, to avoid mutation
    df_train = tv_df.copy()
    df_test = test_df.copy()
    
    # apply label encoding (which Scikit-Learn models require, but *Prophets don't)
    le_dict, tv_df = label_encoder(df_train) # should leave broader scope's tv_df alone
    _, test_df = label_encoder(df_test) # should leave broader scope's test_df alone
    del df_train, df_test
    
    residual_tv_df = pd.DataFrame({
        'date': orig_train_df['date'],
        'num_sold': orig_train_df['num_sold']
    })
    print(len(residual_tv_df))
    
    residual_test_df = pd.DataFrame({
        'date': orig_test_df['date'],
    })
    
    # getting rid of unneeded 'target' feature, also num_sold since we're only interested in predicting the residual
    # following @ambrosm lightgbm nb and leaving date in for GroupKFold, but unsure of this (or if I should drop it later)
    tv_df = tv_df.drop(columns=['target', 'num_sold'])#, 'date'])
    test_df = test_df.drop(columns=['target', 'num_sold'])#, 'date'])
    
    kfolds= GroupKFold(n_splits=4)
    
    test_fold_preds = {}
    
    for forecast_model in forecast_models:
        print(f"Working with forecasts from {forecast_model}...")
        print("-----------------------------------------------------")
        forecast = tv_forecast_preds[forecast_model] # pull out the predictions on the t-v sets for a given estimator
        residuals = residual_tv_df['num_sold'] - forecast # get the residuals for the given model's forecast
#         residual_df[f'actual_{forecast_model}_residual'] = residuals # may not need to put this in there
#         tv_df['residual'] = residuals # residuals will rotate in and out of this feature
        
        
        X_test = test_df.drop(columns=['date'])
#         print("y.shape is ", y.shape)
        
        # prepare for Group K-Fold cross-val; below from @ambrosm LightGBM notebook
        oof_preds = pd.Series(0, index=tv_df.index)
#         test_preds_df = pd.DataFrame({
#             'date': test_df['date']
#         })
        score_list = []
#         params['seed'] = 1
        for fold, (train_idx, val_idx) in enumerate(kfolds.split(tv_df, groups=tv_df.date.dt.year)):
            print("-----------------------------------------------------")
            print(f"FOLD {fold}")
            X = tv_df.iloc[train_idx].drop(columns=['date'])
            y = residuals.iloc[train_idx]#['date']
            X_valid = tv_df.iloc[val_idx].drop(columns=['date'])
            y_valid = residuals.iloc[val_idx]#['date']
            
            model = estimator(**model_kwargs)
            model.fit(X, y)
            
            residual_valid_preds = model.predict(X_valid)
            residual_test_preds = model.predict(X_test)
            
            oof_preds[val_idx] = residual_valid_preds
#             test_preds_df[f'{forecast_model}_fold_{fold}_preds'] = residual_test_preds
#             residual_test_df = residual_test_df.join(test_preds_df)
#             smape = SMAPE(y_pred=residual_valid_preds, y_true=y_valid.values)
#             print(f"SMAPE: {smape}")
            rmse = math.sqrt(mean_squared_error(y_pred=residual_valid_preds, y_true=y_valid.values))
            print(f"RMSE: {rmse}")
            test_fold_preds[f'{forecast_model}_{fold}_residual_preds'] = residual_test_preds
        residual_tv_df[f'{forecast_model}_oof_residual_preds'] = oof_preds
#         residual_test_df = residual_test_df.join(test_preds_df)
    return residual_tv_df, test_fold_preds #residual_test_df
        
            
            
#             model, smape = fit_model(X_tr, X_va, run=0, fold=fold)

# print(f"Average SMAPE: {sum(score_list) / len(score_list):.5f}")
# with open('oof.pickle', 'wb') as handle: pickle.dump(oof, handle)
#         X = tv_df.drop(columns=['residual'])
#         y = tv_df['residual']
        
#         tv_
    
    
    
        
#     y_valid = valid_df['num_sold']
#     valid_df = valid_df.drop(columns=['target'])

In [81]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

#### Residual model hyperparams

In [84]:
xgboost_params = {
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'eval_metric': ['mae', 'mape', 'rmse'],
    'learning_rate': .09,
    'max_depth': 0,
    'subsample': .15,
    'sampling_method': 'gradient_based',
    'seed': 42,
    'grow_policy': 'lossguide',
    'max_leaves': 255,
    'lambda': 100,
#     'n_estimators': 3000,
    'objective': 'reg:squarederror',
    'n_estimators': 500
#     'verbose': True,
}


lightgbm_params = {
    'objective': 'mse',
    'random_state': 42,
    'device_type': 'cpu',
    'n_jobs': -1,
#                 eval_metric='auc',
#     'device_type': 'gpu',
#     'max_bin': 63, # 15 might be even better for GPU perf, but depends on dataset -- see https://lightgbm.readthedocs.io/en/latest/GPU-Performance.html
#     'gpu_use_dp': False,
    'max_depth': 0,
    'learning_rate': 0.1,
    'subsample': .15,
    'n_estimators': 1500,
}

catboost_params = {
    'task_type':'GPU',
    'silent':True,
    'random_state':42,
}
                

XGBoost SMAPEs with `n_estimators=500`:

```
26298
Working with forecasts from prophet...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 44.22169948290956
-----------------------------------------------------
FOLD 1
RMSE: 42.2884194618039
-----------------------------------------------------
FOLD 2
RMSE: 38.87851134388903
-----------------------------------------------------
FOLD 3
RMSE: 39.14401892347087
Working with forecasts from neuralprophet...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 71.59711327241503
-----------------------------------------------------
FOLD 1
RMSE: 50.410812085025206
-----------------------------------------------------
FOLD 2
RMSE: 56.30623610755547
-----------------------------------------------------
FOLD 3
RMSE: 71.87002649034811
Working with forecasts from ridge...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 29.951196797722172
-----------------------------------------------------
FOLD 1
RMSE: 33.87710734178301
-----------------------------------------------------
FOLD 2
RMSE: 30.3425528497887
-----------------------------------------------------
FOLD 3
RMSE: 30.908377053367243
Working with forecasts from linear...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 25.909733767351145
-----------------------------------------------------
FOLD 1
RMSE: 32.55539418287357
-----------------------------------------------------
FOLD 2
RMSE: 28.02573800531867
-----------------------------------------------------
FOLD 3
RMSE: 27.512157750399876
Working with forecasts from huber...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 43.8980188030959
-----------------------------------------------------
FOLD 1
RMSE: 55.40375581530791
-----------------------------------------------------
FOLD 2
RMSE: 46.37734856211295
-----------------------------------------------------
FOLD 3
RMSE: 41.743947334147386
Working with forecasts from lasso...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 42.212939444847706
-----------------------------------------------------
FOLD 1
RMSE: 58.6828784233263
-----------------------------------------------------
FOLD 2
RMSE: 47.91103516420516
-----------------------------------------------------
FOLD 3
RMSE: 41.89282012115221
```

#### Residual model training / loading

In [85]:
%time 
xgb_tv_df, xgb_test_fold_preds = residual_trainer(estimator=XGBRegressor, model_kwargs=xgboost_params,)
dump(xgb_tv_df, predpath/'20220125_+earth_residual_oof_residual_xgboost_preds.joblib')
dump(xgb_test_fold_preds, predpath/'20220125_+earth_residual_test_residual_xgboost_preds.joblib')

# loading the 500-estimator version, which is universally better
# xgb_tv_df = load(predpath/'20220124_residual_oof_xgboost_preds.joblib')
# xgb_test_fold_preds = load(predpath/'20220124_residual_test_xgboost_preds.joblib')

CPU times: user 0 ns, sys: 3 µs, total: 3 µs
Wall time: 6.68 µs
26298
Working with forecasts from prophet...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 44.22169948290956
-----------------------------------------------------
FOLD 1
RMSE: 42.2884194618039
-----------------------------------------------------
FOLD 2
RMSE: 38.87851134388903
-----------------------------------------------------
FOLD 3
RMSE: 39.14401892347087
Working with forecasts from neuralprophet...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 72.2965394109592
-----------------------------------------------------
FOLD 1
RMSE: 50.48678606565055
-----------------------------------------------------
FOLD 2
RMSE: 56.39589310004499
-----------------------------------------------------
FOLD 3
RMSE: 71.94274727206121
Working with forecasts from ridge...
-----------------------

['/media/sf/easystore/kaggle_data/tabular_playgrounds/jan2022/preds/20220125_+earth_residual_test_residual_xgboost_preds.joblib']

In [86]:
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [87]:
%time
lgb_tv_df, lgb_test_fold_preds = residual_trainer(estimator=LGBMRegressor, model_kwargs=lightgbm_params,)
dump(lgb_tv_df, predpath/'20220125_+earth_residual_oof_residual_lightgbm_preds.joblib')
dump(lgb_test_fold_preds, predpath/'20220125_+earth_residual_test_residual_lightgbm_preds.joblib')
# lgb_tv_df = load(predpath/'20220124_residual_oof_residual_lightgbm_preds.joblib')
# lgb_test_fold_preds = load(predpath/'20220124_residual_test_residual_lightgbm_preds.joblib')

CPU times: user 0 ns, sys: 34 µs, total: 34 µs
Wall time: 4.77 µs
26298
Working with forecasts from prophet...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 40.2958973133468
-----------------------------------------------------
FOLD 1
RMSE: 40.72754418203894
-----------------------------------------------------
FOLD 2
RMSE: 34.289662190196665
-----------------------------------------------------
FOLD 3
RMSE: 35.118251779330976
Working with forecasts from neuralprophet...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 71.44951581377815
-----------------------------------------------------
FOLD 1
RMSE: 47.489823189781696
-----------------------------------------------------
FOLD 2
RMSE: 53.766625686957966
-----------------------------------------------------
FOLD 3
RMSE: 58.62494371250313
Working with forecasts from ridge...
----------------

['/media/sf/easystore/kaggle_data/tabular_playgrounds/jan2022/preds/20220125_+earth_residual_test_residual_lightgbm_preds.joblib']

In [88]:
%time
cat_tv_df, cat_test_fold_preds = residual_trainer(estimator=CatBoostRegressor, model_kwargs=catboost_params,)
dump(cat_tv_df, predpath/'20220125_+earth_residual_oof_residual_catboost_preds.joblib')
dump(cat_test_fold_preds, predpath/'20220125_+earth_residual_test_residual_catboost_preds.joblib')
# cat_tv_df = load(predpath/'20220124_residual_oof_residual_catboost_preds.joblib')
# cat_test_fold_preds = load(predpath/'20220124_residual_test_residual_catboost_preds.joblib')

CPU times: user 34 µs, sys: 1e+03 ns, total: 35 µs
Wall time: 4.77 µs
26298
Working with forecasts from prophet...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 34.85084358418546
-----------------------------------------------------
FOLD 1
RMSE: 39.134690709805604
-----------------------------------------------------
FOLD 2
RMSE: 32.316241553438786
-----------------------------------------------------
FOLD 3
RMSE: 32.297408734705535
Working with forecasts from neuralprophet...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 64.4820465743269
-----------------------------------------------------
FOLD 1
RMSE: 45.80180130203257
-----------------------------------------------------
FOLD 2
RMSE: 47.036335391797884
-----------------------------------------------------
FOLD 3
RMSE: 54.06048073783771
Working with forecasts from ridge...
------------

['/media/sf/easystore/kaggle_data/tabular_playgrounds/jan2022/preds/20220125_+earth_residual_test_residual_catboost_preds.joblib']

In [89]:
%time
# ridge_tv_df, ridge_test_fold_preds = residual_trainer(estimator=Ridge)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [90]:
%time
# lasso_tv_df, lasso_test_fold_preds = residual_trainer(estimator=Lasso)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


In [91]:
%time
# linear_tv_df, linear_test_fold_preds = residual_trainer(estimator=LinearRegression)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


In [92]:
# dump(valid_forecast_preds, predpath/'20220124_forecast_valid2018_preds.joblib')
# dump(test_forecast_preds, predpath/'20220124_forecast_test_preds.joblib')

#### Assembling together residual predictions

In [93]:
tv_dfs = {
    'xgb': xgb_tv_df, 
    'lgb': lgb_tv_df, 
    'cat': cat_tv_df,
#     'ridge': ridge_tv_df,
#     'lasso': lasso_tv_df,
#     'linear': linear_tv_df,
}

for arch in tv_dfs.keys():
    for forecast_model in forecast_models:
        tv_dfs[arch][f'{forecast_model}_pred'] = tv_preds[forecast_model]
        tv_dfs[arch][f'{forecast_model}_residual'] = tv_dfs[arch]['num_sold'] - tv_preds[forecast_model] #- xgb_tv_df['num_sold']

In [94]:
# xgb_tv_df

In [97]:
tv_dfs['xgb']

,date,num_sold,prophet_oof_residual_preds,neuralprophet_oof_residual_preds,ridge_oof_residual_preds,linear_oof_residual_preds,huber_oof_residual_preds,lasso_oof_residual_preds,earth_oof_residual_preds,prophet_pred,...,ridge_pred,ridge_residual,linear_pred,linear_residual,huber_pred,huber_residual,lasso_pred,lasso_residual,earth_pred,earth_residual
0,2015-01-01,329,1.278618,19.981554,34.222641,3.645102,139.376175,166.224564,14.680798,346.560416,...,283.272258,45.727742,324.188332,4.811668,239.868861,89.131139,176.381725,152.618275,360.672190,-31.672190
1,2015-01-01,520,-10.982243,35.412369,57.832901,-9.185784,192.085785,217.624084,12.078963,536.203586,...,439.771734,80.228266,503.081569,16.918431,334.618687,185.381313,334.453055,185.546945,584.851806,-64.851806
2,2015-01-01,146,-5.641450,3.151324,11.319877,-6.932339,66.705956,53.481075,-3.095822,143.412803,...,120.885683,25.114317,136.667932,9.332068,96.086063,49.913937,89.802390,56.197610,138.401176,7.598824
3,2015-01-01,572,7.359212,25.241949,65.995567,11.750156,285.283234,283.822510,56.345188,590.117165,...,488.810710,83.189290,555.531957,16.468043,330.014790,241.985210,307.788806,264.211194,562.479785,9.520215
4,2015-01-01,911,0.859362,103.271629,132.802261,-6.488067,122.903603,407.049622,8.930732,939.673009,...,771.701702,139.298298,890.469201,20.530799,875.922092,35.077908,583.510103,327.489897,944.584508,-33.584508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26293,2018-12-31,823,-3.253311,283.321930,132.241791,-10.704310,465.822357,427.108459,-25.774075,898.322121,...,723.714830,99.285170,846.597603,-23.597603,407.206130,415.793870,397.285794,425.714206,836.947940,-13.947940
26294,2018-12-31,250,-14.439471,6.964965,46.179234,5.300345,118.407372,119.623322,-0.278643,253.512355,...,205.880079,44.119921,241.048951,8.951049,132.620712,117.379288,126.765608,123.234392,254.287204,-4.287204
26295,2018-12-31,1004,-31.400349,364.578430,146.736877,-6.179089,552.454895,531.086609,-23.914564,1039.635205,...,832.192362,171.807638,975.785339,28.214661,401.224848,602.775152,430.368706,573.631294,1058.174059,-54.174059
26296,2018-12-31,1441,34.119045,638.341980,227.119736,-13.130022,642.822388,774.511536,-40.309673,1526.908216,...,1255.885410,185.114590,1468.776593,-27.776593,863.230649,577.769351,689.775798,751.224202,2036.549357,-595.549357


In [98]:
for arch in tv_dfs.keys(): # xgb, then lgb, then cat
    print(f"\nFor {arch}...\n")
    for forecast_model in forecast_models:
        
#         if arch == 'xgb':
#             adjusted_smape = SMAPE(y_pred=tv_dfs[arch][f'{forecast_model}_pred']+tv_dfs[arch][f'{forecast_model}_oof_preds'], y_true=tv_dfs[arch]['num_sold'])
#         else:
        adjusted_smape = SMAPE(y_pred=tv_dfs[arch][f'{forecast_model}_pred']+tv_dfs[arch][f'{forecast_model}_oof_residual_preds'], y_true=tv_dfs[arch]['num_sold'])
        original_smape = SMAPE(y_pred=tv_dfs[arch][f'{forecast_model}_pred'], y_true=tv_dfs[arch]['num_sold'])
        print(f'Before residual adjustment, SMAPE for {forecast_model} was {original_smape}. Final SMAPE for {forecast_model} is {adjusted_smape}.')


For xgb...

Before residual adjustment, SMAPE for prophet was 7.098824805150043. Final SMAPE for prophet is 6.459337760847558.
Before residual adjustment, SMAPE for neuralprophet was 7.926662675729486. Final SMAPE for neuralprophet is 9.055081593920645.
Before residual adjustment, SMAPE for ridge was 4.781041939812455. Final SMAPE for ridge is 5.4770486671076055.
Before residual adjustment, SMAPE for linear was 4.175573461450916. Final SMAPE for linear is 5.141834427310028.
Before residual adjustment, SMAPE for huber was 14.396222549610735. Final SMAPE for huber is 7.442538563573811.
Before residual adjustment, SMAPE for lasso was 15.11713989060885. Final SMAPE for lasso is 7.000306029463663.
Before residual adjustment, SMAPE for earth was 5.338351055481194. Final SMAPE for earth is 6.105282898894313.

For lgb...

Before residual adjustment, SMAPE for prophet was 7.098824805150043. Final SMAPE for prophet is 5.977672631338595.
Before residual adjustment, SMAPE for neuralprophet was 7.

With + as operator:
```
Before residual adjustment, SMAPE for prophet was 7.098824805150043. Final SMAPE for prophet is 14.394425598661853.
Before residual adjustment, SMAPE for neuralprophet was 7.930322204823288. Final SMAPE for neuralprophet is 17.10022501507438.
Before residual adjustment, SMAPE for ridge was 4.781041939812455. Final SMAPE for ridge is 9.6272249020869.
Before residual adjustment, SMAPE for linear was 4.175573461450916. Final SMAPE for linear is 8.315020784789288.
Before residual adjustment, SMAPE for huber was 14.396222549610735. Final SMAPE for huber is 30.438323388261274.
Before residual adjustment, SMAPE for lasso was 15.11713989060885. Final SMAPE for lasso is 31.73839575442759.
```

In [99]:
xgb_test_preds_df = pd.DataFrame(xgb_test_fold_preds)
lgb_test_preds_df = pd.DataFrame(lgb_test_fold_preds)
cat_test_preds_df = pd.DataFrame(cat_test_fold_preds)

test_dfs_dict = {
    'xgb': xgb_test_preds_df,
    'lgb': lgb_test_preds_df,
    'cat': cat_test_preds_df,
}

In [100]:
xgb_test_preds_df

,prophet_0_residual_preds,prophet_1_residual_preds,prophet_2_residual_preds,prophet_3_residual_preds,neuralprophet_0_residual_preds,neuralprophet_1_residual_preds,neuralprophet_2_residual_preds,neuralprophet_3_residual_preds,ridge_0_residual_preds,ridge_1_residual_preds,...,huber_2_residual_preds,huber_3_residual_preds,lasso_0_residual_preds,lasso_1_residual_preds,lasso_2_residual_preds,lasso_3_residual_preds,earth_0_residual_preds,earth_1_residual_preds,earth_2_residual_preds,earth_3_residual_preds
0,1.628039,-22.776003,-4.337413,9.217171,-1.113973,-14.270683,9.624284,20.024666,40.829250,32.131512,...,117.732880,152.466690,171.862076,167.369675,165.736267,189.338898,3.466743,-5.279877,4.872921,15.645398
1,-1.307783,-29.234877,-19.299370,-4.380471,38.565910,21.016108,30.840876,49.329002,76.017052,67.138077,...,180.163071,218.693390,227.298630,220.408157,213.388229,245.253067,4.695372,-7.733302,-15.240776,17.300634
2,0.548766,-9.147127,-2.938624,-1.790694,-1.135752,-8.805696,3.665533,5.309555,17.624104,17.756908,...,67.835770,82.093063,67.499695,60.452721,63.432430,64.438507,4.077691,-8.597854,-6.490247,1.034713
3,11.687498,-30.114052,6.320381,14.146542,14.123075,-9.601943,12.965332,26.437778,81.154427,64.653954,...,288.372559,320.592285,301.413605,306.337585,279.525238,316.741058,62.406052,40.480110,59.990955,72.885109
4,-18.396700,-8.923676,6.745495,-4.918637,81.434113,111.132454,113.018555,98.424866,114.582291,143.575012,...,91.613174,134.632111,415.758423,449.545532,398.279816,440.636688,0.899182,17.500675,12.615296,14.708526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,-100.418335,-32.443649,-59.012535,-8.907444,139.995728,198.155090,168.623062,276.320374,105.135254,117.854515,...,370.991180,414.747345,354.354706,386.669739,401.603943,436.174225,0.590873,-23.707037,-44.231205,-75.286507
6566,-29.682568,-29.894447,-19.465675,-14.519586,-19.383842,-28.602324,-16.461973,19.153826,36.606209,35.228756,...,110.502045,99.124031,110.722855,102.684364,112.017014,107.182640,7.660434,2.207999,14.715510,-1.407388
6567,-87.339149,-49.978775,-58.895260,14.599092,230.109726,267.625427,238.597366,347.494507,147.598892,139.108032,...,523.964050,589.809082,484.957825,499.436707,526.032043,545.696228,-9.219683,-6.820998,-10.478740,-35.574120
6568,-71.144836,6.305995,-51.087128,34.664040,456.211670,523.982727,444.451630,567.523499,212.100479,217.488190,...,519.880859,593.520569,658.354858,731.390320,701.821655,754.522644,-51.764935,-23.280802,-131.861771,-192.554993


In [101]:
lgb_test_preds_df

,prophet_0_residual_preds,prophet_1_residual_preds,prophet_2_residual_preds,prophet_3_residual_preds,neuralprophet_0_residual_preds,neuralprophet_1_residual_preds,neuralprophet_2_residual_preds,neuralprophet_3_residual_preds,ridge_0_residual_preds,ridge_1_residual_preds,...,huber_2_residual_preds,huber_3_residual_preds,lasso_0_residual_preds,lasso_1_residual_preds,lasso_2_residual_preds,lasso_3_residual_preds,earth_0_residual_preds,earth_1_residual_preds,earth_2_residual_preds,earth_3_residual_preds
0,30.659798,-5.406039,25.685431,25.702927,38.625799,-26.543532,42.386490,41.566642,62.010359,38.643447,...,156.201496,165.882975,203.273303,166.865059,204.484263,206.633983,14.030557,7.889509,23.599155,37.764264
1,23.326129,-14.781889,7.569961,14.815576,74.046034,-2.021053,72.035079,59.873683,86.575967,59.412709,...,245.791512,271.419730,274.603292,189.301795,253.664893,277.652861,0.009271,24.333671,-6.616261,-0.398344
2,14.903635,9.096575,19.581296,16.245287,17.059769,-4.708975,18.002370,15.954930,28.641066,15.127044,...,82.106209,87.898212,75.172092,61.772996,77.285651,71.062885,8.560491,1.254045,12.916000,17.966165
3,32.888279,-13.094183,39.609446,44.406229,39.773920,-34.176598,61.832559,54.490520,99.549027,55.405590,...,340.399942,346.054952,321.728073,255.686679,326.044949,327.006351,68.726943,36.798843,82.212348,88.371916
4,-3.216878,-10.839813,27.319067,10.627042,125.004497,63.631370,149.099507,117.800786,124.353946,114.335094,...,148.752113,144.124544,445.731385,345.544891,455.457338,435.547905,-8.894213,24.726179,3.853427,-16.385015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,-68.673045,-56.764350,-62.810567,1.022159,180.027155,255.176557,147.445339,248.247112,120.124596,137.676273,...,379.192802,431.717952,376.910966,384.656657,387.463713,413.815027,-2.797492,-17.258207,-38.465439,-50.397515
6566,-5.854447,-6.348483,-13.550638,7.160769,11.840255,27.455167,18.720853,14.228303,46.428923,49.226229,...,112.660298,127.334350,129.797733,129.195569,124.898281,131.851480,9.458203,6.667925,16.496136,3.295281
6567,-42.575155,-77.468770,-69.646457,14.660957,298.763778,315.009559,218.779005,354.199629,169.165384,156.205672,...,526.079526,604.837679,508.729383,505.244678,511.136612,537.605524,-4.910637,-16.472856,26.800664,-0.590565
6568,-74.562045,-34.338660,-65.610515,36.533271,508.040128,569.731072,394.333840,553.090571,220.473254,231.219097,...,502.212720,565.484246,672.356991,709.551573,644.943501,708.009935,-67.639513,-28.942641,-64.581800,-136.314135


## Submission of Naive Forecasts

Something's up with the residuals; for now, let's just run a simple model on the original forecasts.

In [102]:
model = Lasso()
# model = Ridge() # not good
# model = LinearRegression() # not good
# model = CatBoostRegressor(**catboost_params) # very bad

In [103]:
tv_preds

,date,num_sold,prophet,neuralprophet,ridge,linear,huber,lasso,earth
0,2015-01-01,329,346.560416,329.134521,283.272258,324.188332,239.868861,176.381725,360.672190
1,2015-01-01,520,536.203586,458.008301,439.771734,503.081569,334.618687,334.453055,584.851806
2,2015-01-01,146,143.412803,145.325577,120.885683,136.667932,96.086063,89.802390,138.401176
3,2015-01-01,572,590.117165,552.571167,488.810710,555.531957,330.014790,307.788806,562.479785
4,2015-01-01,911,939.673009,781.967285,771.701702,890.469201,875.922092,583.510103,944.584508
...,...,...,...,...,...,...,...,...,...
26293,2018-12-31,823,898.322121,669.418457,723.714830,846.597603,407.206130,397.285794,836.947940
26294,2018-12-31,250,253.512355,227.158142,205.880079,241.048951,132.620712,126.765608,254.287204
26295,2018-12-31,1004,1039.635205,715.639648,832.192362,975.785339,401.224848,430.368706,1058.174059
26296,2018-12-31,1441,1526.908216,980.234009,1255.885410,1468.776593,863.230649,689.775798,2036.549357


In [104]:
X = tv_preds.drop(columns=['num_sold'])
y = tv_preds['num_sold']

In [105]:
test_forecast_preds

,date,prophet,neuralprophet,ridge,linear,huber,lasso,earth
0,2019-01-01,402.296576,383.543457,359.817081,392.328390,248.728792,221.118411,417.102143
1,2019-01-01,631.642099,551.044983,560.783324,620.092467,347.337935,347.623611,693.844970
2,2019-01-01,169.992094,173.065674,153.552940,166.788184,99.483226,112.590591,181.642807
3,2019-01-01,684.559821,648.783691,622.237644,675.674104,342.549613,384.790895,594.166542
4,2019-01-01,1092.875362,965.683716,969.065554,1084.980801,911.102831,601.189400,1142.564632
...,...,...,...,...,...,...,...,...
6565,2019-12-31,917.085861,674.891174,742.331990,797.964290,404.876685,391.134492,830.606248
6566,2019-12-31,261.569590,232.082169,210.920805,225.576610,131.774790,129.930008,269.498973
6567,2019-12-31,1065.400121,730.457275,852.091174,925.788409,398.926697,440.077496,1133.560974
6568,2019-12-31,1568.407214,1005.392944,1286.814471,1397.071983,858.595973,677.887045,2627.470943


In [106]:
X_test = test_forecast_preds

In [107]:
X['date'] = pd.to_datetime(X.date)

In [108]:
X['date'] = X['date'].map(dt.datetime.toordinal)

In [ ]:
# X = X.drop(columns=['huber'])

In [109]:
model.fit(X,y)

Lasso()

In [ ]:
# abs(model.coef_)

In [110]:
X_test

,date,prophet,neuralprophet,ridge,linear,huber,lasso,earth
0,2019-01-01,402.296576,383.543457,359.817081,392.328390,248.728792,221.118411,417.102143
1,2019-01-01,631.642099,551.044983,560.783324,620.092467,347.337935,347.623611,693.844970
2,2019-01-01,169.992094,173.065674,153.552940,166.788184,99.483226,112.590591,181.642807
3,2019-01-01,684.559821,648.783691,622.237644,675.674104,342.549613,384.790895,594.166542
4,2019-01-01,1092.875362,965.683716,969.065554,1084.980801,911.102831,601.189400,1142.564632
...,...,...,...,...,...,...,...,...
6565,2019-12-31,917.085861,674.891174,742.331990,797.964290,404.876685,391.134492,830.606248
6566,2019-12-31,261.569590,232.082169,210.920805,225.576610,131.774790,129.930008,269.498973
6567,2019-12-31,1065.400121,730.457275,852.091174,925.788409,398.926697,440.077496,1133.560974
6568,2019-12-31,1568.407214,1005.392944,1286.814471,1397.071983,858.595973,677.887045,2627.470943


In [111]:
# X_test['date'] = X['date'].map(dt.datetime.toordinal)
X_test['date'] = pd.to_datetime(X_test.date)

In [112]:
X_test

,date,prophet,neuralprophet,ridge,linear,huber,lasso,earth
0,2019-01-01,402.296576,383.543457,359.817081,392.328390,248.728792,221.118411,417.102143
1,2019-01-01,631.642099,551.044983,560.783324,620.092467,347.337935,347.623611,693.844970
2,2019-01-01,169.992094,173.065674,153.552940,166.788184,99.483226,112.590591,181.642807
3,2019-01-01,684.559821,648.783691,622.237644,675.674104,342.549613,384.790895,594.166542
4,2019-01-01,1092.875362,965.683716,969.065554,1084.980801,911.102831,601.189400,1142.564632
...,...,...,...,...,...,...,...,...
6565,2019-12-31,917.085861,674.891174,742.331990,797.964290,404.876685,391.134492,830.606248
6566,2019-12-31,261.569590,232.082169,210.920805,225.576610,131.774790,129.930008,269.498973
6567,2019-12-31,1065.400121,730.457275,852.091174,925.788409,398.926697,440.077496,1133.560974
6568,2019-12-31,1568.407214,1005.392944,1286.814471,1397.071983,858.595973,677.887045,2627.470943


In [113]:
X_test['date'] = X_test['date'].map(dt.datetime.toordinal)
# X_test = X_test.drop(columns=['huber'])

In [114]:
final_test_preds = model.predict(X_test)

In [115]:
final_tv_preds = model.predict(X)

So, with all of the model preds used, Lasso SMAPE is 4.15283882859097
- Dropping Huber only, it's 4.152469194333937
- Dropping Huber+Lasso it's 4.1525522971210345
- Dropping Lasso only, it's 4.15285802958327

So of these, definitely the best is dropping Huber only.

With all the model preds used, Ridge SMAPE is 4.158465882203197 -- that's much worse.

With all the model preds used, LinearRegression SMAPE is 4.158465902939169 -- also much worse.

Using the default hyperparams for CatBoost is even worse: 4.31463984301381

In [116]:
SMAPE(y_pred=final_tv_preds, y_true=y)

4.1531645410329565

SMAPE on all of them, with Lasso as final, Earth included, is 4.1531645410329565.

In [117]:
sample_df = pd.read_csv(datapath/'sample_submission.csv')

In [118]:
sample_df.loc[:, 'num_sold'] = final_test_preds

In [119]:
sample_df

,row_id,num_sold
0,26298,382.387610
1,26299,607.651600
2,26300,159.829226
3,26301,661.969460
4,26302,1063.321727
...,...,...
6565,32863,787.496001
6566,32864,217.033343
6567,32865,915.852977
6568,32866,1391.866271


In [120]:
sample_df['num_sold'] = sample_df['num_sold'].apply(round)

In [121]:
sample_df.head()

,row_id,num_sold
0,26298,382
1,26299,608
2,26300,160
3,26301,662
4,26302,1063


In [122]:
sample_df.to_csv(subpath/f"20220125_forecasts+earth_lasso_preds_rounded.csv", index=False)


## Submission of Adjusted XGB Naivish

In [124]:
# oof_features = ['']
adjusted_oof_preds = pd.DataFrame({
    'date': orig_train_df['date'],
})

tv_dfs_dict = {
    'xgb': xgb_tv_df,
    'lgb': lgb_tv_df,
    'cat': cat_tv_df,
}

for arch in tv_dfs_dict.keys():
    print(f"{arch}\n")
    for forecast_model in forecast_models:
#         if arch == 'xgb':
#                         adjusted_oof_preds[f"{arch}+{forecast_model}"] = tv_dfs_dict[arch][f'{forecast_model}_pred']+tv_dfs_dict[arch][f'{forecast_model}_oof_preds']
#         else:
        adjusted_oof_preds[f"{arch}+{forecast_model}"] = tv_dfs_dict[arch][f'{forecast_model}_pred']+tv_dfs_dict[arch][f'{forecast_model}_oof_residual_preds']

xgb

lgb

cat



In [125]:
adjusted_oof_preds

,date,xgb+prophet,xgb+neuralprophet,xgb+ridge,xgb+linear,xgb+huber,xgb+lasso,xgb+earth,lgb+prophet,lgb+neuralprophet,...,lgb+huber,lgb+lasso,lgb+earth,cat+prophet,cat+neuralprophet,cat+ridge,cat+linear,cat+huber,cat+lasso,cat+earth
0,2015-01-01,347.839034,349.116076,317.494899,327.833433,379.245036,342.606288,375.352988,335.467823,332.675307,...,354.488671,333.410991,385.836067,333.915969,328.715831,314.826055,317.206583,346.279165,336.233347,381.225120
1,2015-01-01,525.221343,493.420670,497.604635,493.895785,526.704472,552.077139,596.930769,519.648545,473.685888,...,492.573564,541.304068,590.518820,509.140981,494.509681,506.135450,495.753502,495.232955,522.987775,589.324197
2,2015-01-01,137.771353,148.476901,132.205560,129.735593,162.792018,143.283465,135.305354,148.667331,138.844205,...,154.851398,136.278547,145.985095,142.338521,127.459581,128.473396,133.452584,145.584398,133.919333,141.192906
3,2015-01-01,597.476377,577.813116,554.806277,567.282113,615.298024,591.611316,618.824973,590.189094,556.542029,...,593.972827,571.047420,615.613465,583.211954,628.725821,569.523275,547.732239,615.549186,636.556795,613.489740
4,2015-01-01,940.532371,885.238914,904.503964,883.981135,998.825694,990.559725,953.515239,921.358231,850.556192,...,920.105059,926.419508,932.330567,935.543658,947.868121,898.809221,881.201439,964.717169,1008.016399,958.648482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26293,2018-12-31,895.068810,952.740387,855.956621,835.893293,873.028487,824.394254,811.173865,841.557771,924.595014,...,791.482009,781.942451,819.689733,927.389893,968.623145,854.088327,848.968382,811.147037,832.155634,834.802035
26294,2018-12-31,239.072883,234.123107,252.059312,246.349297,251.028084,246.388929,254.008561,247.163872,254.613309,...,250.723332,255.961176,260.955129,216.592747,265.774641,250.231972,244.564056,223.426179,236.455523,255.052840
26295,2018-12-31,1008.234856,1080.218079,978.929240,969.606250,953.679743,961.455315,1034.259495,962.166435,1030.649207,...,959.214436,935.613384,1041.701203,1075.349456,1073.352314,971.048282,969.551301,900.275331,944.275194,1027.932925
26296,2018-12-31,1561.027261,1618.575989,1483.005146,1455.646571,1506.053037,1464.287333,1996.239684,1492.569556,1549.965081,...,1387.095641,1399.327370,2007.606717,1663.235599,1589.577839,1468.298331,1468.509798,1504.615944,1418.220415,2036.327679


In [126]:
adjusted_oof_preds['num_sold'] = orig_train_df['num_sold']

In [127]:
model = Lasso()

In [128]:
X = adjusted_oof_preds.drop(columns=['num_sold'])
y = adjusted_oof_preds['num_sold']

In [129]:
adjusted_test_preds = pd.DataFrame({
    'date': orig_test_df['date']
})

# tv_dfs_dict = {
#     'xgb': xgb_tv_df,
#     'lgb': lgb_tv_df,
#     'cat': cat_tv_df,
# }

for arch in test_dfs_dict.keys():#['xgb', 'lgb', 'cat']: #prophet_0_residual_preds
    for forecast_model in forecast_models:
        adjusted_test_preds[f'{arch}+{forecast_model}'] = sum([test_dfs_dict[arch][f'{forecast_model}_{i}_residual_preds'] for i in range(4)]) / 4
adjusted_test_preds

,date,xgb+prophet,xgb+neuralprophet,xgb+ridge,xgb+linear,xgb+huber,xgb+lasso,xgb+earth,lgb+prophet,lgb+neuralprophet,...,lgb+huber,lgb+lasso,lgb+earth,cat+prophet,cat+neuralprophet,cat+ridge,cat+linear,cat+huber,cat+lasso,cat+earth
0,2019-01-01,-4.067052,3.566073,41.074242,-2.927207,128.356812,173.576721,4.676296,19.160529,24.008850,...,153.611849,195.314152,20.820871,-14.551260,-1.800107,39.869848,-14.322520,132.503743,170.387482,12.443302
1,2019-01-01,-13.555625,34.937973,68.296036,-5.406509,199.275787,226.587021,-0.244518,7.732444,50.983436,...,233.720382,248.805710,4.332084,-25.338150,41.845209,75.834353,-11.974044,202.962403,226.909551,-2.204298
2,2019-01-01,-3.331920,-0.241590,18.091089,-0.690675,70.900955,63.955837,-2.493924,14.956698,11.577024,...,82.048371,71.323406,10.174175,-1.492796,-9.116931,16.343637,-6.888167,68.528241,53.385513,5.400922
3,2019-01-01,0.510092,10.981060,77.541725,-3.008889,286.174194,301.004395,58.940559,25.952443,30.480100,...,323.630889,307.616513,69.027512,-9.812288,63.185425,81.140506,-19.407028,301.357366,333.277065,34.791091
4,2019-01-01,-6.373379,101.002495,131.558319,-14.222021,109.163857,426.055115,11.430920,5.972354,113.884040,...,126.000091,420.570380,0.825095,-9.019371,172.704790,133.047662,-19.296734,108.357982,440.198751,6.929261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,2019-12-31,-50.195488,195.773560,113.345879,-10.379541,375.811951,394.700653,-35.658470,-46.806451,207.724041,...,390.029170,390.711591,-27.229663,-14.822396,259.902562,119.144230,-2.358119,393.269959,416.836879,-25.232470
6566,2019-12-31,-23.390570,-11.323580,38.962296,5.678431,95.540482,108.151726,5.794139,-4.648200,18.061144,...,119.588726,128.935765,8.979386,-39.835409,7.455011,38.123140,4.600495,88.419512,106.974697,-0.656414
6567,2019-12-31,-45.403526,270.956757,150.691833,5.033105,530.443542,514.030701,-15.523385,-43.757356,296.687993,...,556.224723,515.679049,1.206652,18.973191,331.374321,138.763121,3.077579,493.934774,510.334507,-30.920033
6568,2019-12-31,-20.315483,498.042358,209.089203,-17.959795,551.822754,711.522400,-99.865623,-34.494487,506.298903,...,529.687931,683.715500,-74.369522,88.026475,578.599638,199.991247,-8.266427,631.048378,713.978299,-48.918927


In [130]:
test_forecast_preds

,date,prophet,neuralprophet,ridge,linear,huber,lasso,earth
0,737060,402.296576,383.543457,359.817081,392.328390,248.728792,221.118411,417.102143
1,737060,631.642099,551.044983,560.783324,620.092467,347.337935,347.623611,693.844970
2,737060,169.992094,173.065674,153.552940,166.788184,99.483226,112.590591,181.642807
3,737060,684.559821,648.783691,622.237644,675.674104,342.549613,384.790895,594.166542
4,737060,1092.875362,965.683716,969.065554,1084.980801,911.102831,601.189400,1142.564632
...,...,...,...,...,...,...,...,...
6565,737424,917.085861,674.891174,742.331990,797.964290,404.876685,391.134492,830.606248
6566,737424,261.569590,232.082169,210.920805,225.576610,131.774790,129.930008,269.498973
6567,737424,1065.400121,730.457275,852.091174,925.788409,398.926697,440.077496,1133.560974
6568,737424,1568.407214,1005.392944,1286.814471,1397.071983,858.595973,677.887045,2627.470943


In [131]:
dump(adjusted_test_preds, predpath/'20220125_residual_test_predictions+earth_averaged_by_fold_using_GBMs.joblib')

['/media/sf/easystore/kaggle_data/tabular_playgrounds/jan2022/preds/20220125_residual_test_predictions+earth_averaged_by_fold_using_GBMs.joblib']

In [132]:
dump(adjusted_oof_preds, predpath/'20220125_final_oof_predictions_forecast+Earth+GBMs-residuals.joblib')

['/media/sf/easystore/kaggle_data/tabular_playgrounds/jan2022/preds/20220125_final_oof_predictions_forecast+Earth+GBMs-residuals.joblib']

In [133]:
adjusted_test_residual_preds = adjusted_test_preds.copy() # fixing the confusing name

In [141]:
adjusted_test_preds = pd.DataFrame({
    'date': orig_test_df['date']
})

for arch in ['xgb', 'lgb', 'cat']: # ['xgb', 'lgb', 'cat']:
    for forecast_model in forecast_models:
        adjusted_test_preds[f'{arch}+{forecast_model}'] = test_forecast_preds[f'{forecast_model}'] + adjusted_test_residual_preds[f'{arch}+{forecast_model}']

In [142]:
adjusted_test_preds

,date,xgb+prophet,xgb+neuralprophet,xgb+ridge,xgb+linear,xgb+huber,xgb+lasso,xgb+earth,lgb+prophet,lgb+neuralprophet,...,lgb+huber,lgb+lasso,lgb+earth,cat+prophet,cat+neuralprophet,cat+ridge,cat+linear,cat+huber,cat+lasso,cat+earth
0,2019-01-01,398.229524,387.109530,400.891323,389.401183,377.085603,394.695132,421.778439,421.457105,407.552307,...,402.340641,416.432563,437.923014,387.745316,381.743350,399.686929,378.005871,381.232535,391.505893,429.545444
1,2019-01-01,618.086474,585.982956,629.079360,614.685957,546.613722,574.210632,693.600452,639.374543,602.028419,...,581.058317,596.429322,698.177055,606.303949,592.890192,636.617677,608.118423,550.300338,574.533162,691.640672
2,2019-01-01,166.660175,172.824084,171.644030,166.097509,170.384181,176.546429,179.148883,184.948792,184.642697,...,181.531597,183.913997,191.816982,168.499298,163.948743,169.896578,159.900016,168.011467,165.976105,187.043729
3,2019-01-01,685.069914,659.764751,699.779369,672.665215,628.723807,685.795290,653.107101,710.512264,679.263792,...,666.180502,692.407408,663.194054,674.747533,711.969117,703.378151,656.267077,643.906978,718.067960,628.957633
4,2019-01-01,1086.501983,1066.686211,1100.623873,1070.758780,1020.266688,1027.244515,1153.995552,1098.847716,1079.567756,...,1037.102922,1021.759780,1143.389727,1083.855991,1138.388506,1102.113216,1065.684067,1019.460813,1041.388151,1149.493893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,2019-12-31,866.890374,870.664734,855.677868,787.584748,780.688636,785.835145,794.947778,870.279411,882.615215,...,794.905856,781.846082,803.376585,902.263466,934.793736,861.476220,795.606171,798.146644,807.971371,805.373778
6566,2019-12-31,238.179020,220.758589,249.883101,231.255040,227.315271,238.081734,275.293112,256.921390,250.143313,...,251.363516,258.865774,278.478359,221.734181,239.537180,249.043946,230.177105,220.194302,236.904705,268.842560
6567,2019-12-31,1019.996594,1001.414032,1002.783008,930.821515,929.370239,954.108197,1118.037589,1021.642765,1027.145268,...,955.151420,955.756545,1134.767626,1084.373312,1061.831596,990.854295,928.865988,892.861471,950.412003,1102.640941
6568,2019-12-31,1548.091731,1503.435303,1495.903674,1379.112188,1410.418727,1389.409444,2527.605319,1533.912727,1511.691847,...,1388.283904,1361.602545,2553.101421,1656.433689,1583.992582,1486.805718,1388.805555,1489.644351,1391.865343,2578.552016


In [143]:
X_test = adjusted_test_preds

In [144]:
X['date'] = pd.to_datetime(X.date)

In [145]:
X['date'] = X['date'].map(dt.datetime.toordinal)
model.fit(X,y) # this is the lasso regressor

Lasso()

In [146]:
# X_test['date'] = X['date'].map(dt.datetime.toordinal)
X_test['date'] = pd.to_datetime(X_test.date)
X_test['date'] = X_test['date'].map(dt.datetime.toordinal)

In [147]:
lasso_test_preds = model.predict(X_test)

In [148]:
lasso_tv_preds = model.predict(X)

Naive SMAPE was 4.152715109788985

Below had been 4.384919101043075 with XGBoost included

In [149]:
SMAPE(y_pred=lasso_tv_preds, y_true=y)

4.389347857315039

Below had been 4.27663983822924 with XGBoost included

In [150]:
ridge = Ridge()
ridge.fit(X,y)
ridge_test_preds = ridge.predict(X_test)
ridge_tv_preds = ridge.predict(X)
SMAPE(y_pred=ridge_tv_preds, y_true=y)


4.254092788311878

In [151]:
sample_df = pd.read_csv(datapath/'sample_submission.csv')

In [152]:
sample_df.loc[:, 'num_sold'] = ridge_test_preds

In [153]:
sample_df.head()

,row_id,num_sold
0,26298,378.586742
1,26299,612.019972
2,26300,159.287186
3,26301,665.461693
4,26302,1076.410400


In [154]:
sample_df.to_csv(subpath/f"20220125_forecasts+Earth-Big3-GBMs-residuals_ridge_preds.csv", index=False)
# sample_df.to_csv(subpath/f"{wandb_config['name']}_3level-X_orig+KMeans8+synth-GBM-stack_ensemble_preds.csv", index=False)

In [ ]:
# sample_df = pd.read_csv(subpath/'LB-4.40223__20220124_forecasts_lasso_preds.csv')

In [155]:
sample_df

,row_id,num_sold
0,26298,378.586742
1,26299,612.019972
2,26300,159.287186
3,26301,665.461693
4,26302,1076.410400
...,...,...
6565,32863,812.698251
6566,32864,231.127064
6567,32865,946.133632
6568,32866,1426.518856


In [156]:
sample_df['num_sold'] = sample_df['num_sold'].apply(round)
sample_df

,row_id,num_sold
0,26298,379
1,26299,612
2,26300,159
3,26301,665
4,26302,1076
...,...,...
6565,32863,813
6566,32864,231
6567,32865,946
6568,32866,1427


In [157]:
sample_df.to_csv(subpath/'20220125_forecasts+earth_ridge_preds_rounded.csv',index=False)